In [ ]:
"""
Author: Kadir Goksel Gunduz
Date: 19:03:2025 
"""
# Requirements
import pygame
import sys
import math
import time
from enum import Enum
import random

In [ ]:
#  3B Formasyon Görevi 
import pygame
import sys
import math
import time
from enum import Enum
import random
import os

# Formation types
class Formation(Enum):
    RANDOM = 0
    LINE = 1
    V_SHAPE = 2
    ARROWHEAD = 3
    
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
GRAY = (200, 200, 200)

# Simulation parameters (modifiable)
NUM_DRONES     = 10  # En az 3 drone
DRONE_SPACING  = 50  # X metre (drone arası mesafe)
MAX_ALTITUDE   = 15  # Z metre (uçuş irtifası)
FORMATION_TIME = 10  # T saniye (formasyon koruma süresi)

# Transition speeds
ALTITUDE_CHANGE_SPEED = 100  # Yükselme/alçalma hızı (metre/saniye)
FORMATION_TRANSITION_SPEED = 100  # Formasyon geçiş hızı (metre/saniye)

# Drone class
class Drone:
    def __init__(self, id, x, y, z=0, drone_img=None):
        self.id = id
        self.x = x
        self.y = y
        self.z = z  # Altitude
        self.target_x = x
        self.target_y = y
        self.target_z = z
        self.base_size = 20
        self.color = (random.randint(100, 200), random.randint(100, 200), random.randint(100, 200))
        
        # Load drone image if provided, otherwise create a fallback
        self.drone_img = drone_img
        if self.drone_img is None:
            self.drone_img = pygame.Surface((self.base_size, self.base_size), pygame.SRCALPHA)
            pygame.draw.circle(self.drone_img, (*self.color, 255), 
                             (self.base_size//2, self.base_size//2), self.base_size//2)
    
    def update(self, dt):
        # Move towards target position
        dx = self.target_x - self.x
        dy = self.target_y - self.y
        dz = self.target_z - self.z
        
        if abs(dx) > 0.1:
            self.x += min(FORMATION_TRANSITION_SPEED * dt, abs(dx)) * (1 if dx > 0 else -1)
        if abs(dy) > 0.1:
            self.y += min(FORMATION_TRANSITION_SPEED * dt, abs(dy)) * (1 if dy > 0 else -1)
        if abs(dz) > 0.1:
            self.z += min(ALTITUDE_CHANGE_SPEED * dt, abs(dz)) * (1 if dz > 0 else -1)
    
    def draw(self, screen, scale_factor=10):
        # Calculate display position (altitude affects size and transparency)
        altitude_factor = 1 + (self.z / MAX_ALTITUDE * 0.5)
        display_size = int(self.base_size * altitude_factor)
        
        # Altitude affects transparency
        alpha = min(255, max(100, int(255 - (self.z / MAX_ALTITUDE * 100))))
        
        # Resize drone image based on altitude
        scaled_img = pygame.transform.scale(self.drone_img, (display_size, display_size))
        
        # Create a surface for the drone with proper transparency
        drone_surface = pygame.Surface((display_size, display_size), pygame.SRCALPHA)
        
        # Apply transparency to the drone image
        for x in range(display_size):
            for y in range(display_size):
                color = scaled_img.get_at((x, y))
                if color.a > 0:  # Only modify non-transparent pixels
                    scaled_img.set_at((x, y), (color.r, color.g, color.b, min(color.a, alpha)))
        
        # Draw drone ID
        font = pygame.font.SysFont('Arial', 10)
        id_text = font.render(str(self.id), True, BLACK)
        scaled_img.blit(id_text, (display_size//2 - 4, display_size//2 - 5))
        
        # Draw altitude indicator
        altitude_text = font.render(f"{int(self.z)}m", True, BLACK)
        screen.blit(altitude_text, (int(self.x) - 10, int(self.y) - display_size - 5))
        
        # Blit the drone to the screen
        screen.blit(scaled_img, (int(self.x) - display_size//2, int(self.y) - display_size//2))
        
        # Draw shadow on the ground
        shadow_size = max(5, int(display_size * 0.5))
        shadow_surface = pygame.Surface((shadow_size, shadow_size), pygame.SRCALPHA)
        shadow_alpha = max(50, 150 - int(self.z * 5))
        pygame.draw.ellipse(shadow_surface, (0, 0, 0, shadow_alpha), 
                         (0, 0, shadow_size, shadow_size//2))
        screen.blit(shadow_surface, (int(self.x) - shadow_size//2, int(self.y) + 20))

# Drone Formation Manager
class FormationManager:
    def __init__(self, num_drones, screen_width, screen_height, drone_img=None):
        self.screen_width = screen_width
        self.screen_height = screen_height
        self.drones = []
        self.current_formation = Formation.RANDOM
        self.target_formation = Formation.RANDOM
        self.formation_start_time = 0
        self.formation_complete = False
        self.mission_complete = False
        self.mission_step = 0
        self.missions = []
        self.drone_img = drone_img
        
        # Create drones in initial random positions on the ground
        base_y = screen_height - 100
        for i in range(num_drones):
            x = random.randint(100, screen_width - 100)
            self.drones.append(Drone(i+1, x, base_y, drone_img=self.drone_img))
    
    def set_formation(self, formation_type):
        self.target_formation = formation_type
        self.formation_complete = False
        self.formation_start_time = 0
        
        center_x = self.screen_width // 2
        center_y = self.screen_height // 2
        
        if formation_type == Formation.RANDOM:
            # Random formation on the ground
            base_y = self.screen_height - 100
            for i, drone in enumerate(self.drones):
                drone.target_x = random.randint(100, self.screen_width - 100)
                drone.target_y = base_y
                
        elif formation_type == Formation.LINE:
            # Line formation
            total_width = (len(self.drones) - 1) * DRONE_SPACING
            start_x = center_x - total_width // 2
            
            for i, drone in enumerate(self.drones):
                drone.target_x = start_x + i * DRONE_SPACING
                drone.target_y = center_y
                
        elif formation_type == Formation.V_SHAPE:
            # V formation
            angle = math.pi / 6  # 30 degrees
            
            for i, drone in enumerate(self.drones):
                if i == 0:  # Lead drone
                    drone.target_x = center_x
                    drone.target_y = center_y - DRONE_SPACING
                else:
                    # Alternate between left and right sides
                    side = 1 if i % 2 == 1 else -1
                    position = (i + 1) // 2
                    
                    drone.target_x = center_x + side * position * DRONE_SPACING * math.sin(angle)
                    drone.target_y = center_y + position * DRONE_SPACING * math.cos(angle)
                    
        elif formation_type == Formation.ARROWHEAD:
            # Arrowhead (Lambda Λ) formation
            angle = math.pi / 4  # 45 degrees
            
            for i, drone in enumerate(self.drones):
                if i == 0:  # Lead drone at the front
                    drone.target_x = center_x
                    drone.target_y = center_y - DRONE_SPACING
                else:
                    # Alternate between left and right sides
                    side = 1 if i % 2 == 1 else -1
                    position = (i + 1) // 2
                    
                    drone.target_x = center_x + side * position * DRONE_SPACING * math.sin(angle)
                    drone.target_y = center_y + position * DRONE_SPACING * math.cos(angle)
    
    def set_altitude(self, altitude):
        for drone in self.drones:
            drone.target_z = altitude
    
    def update(self, dt):
        # Update all drones
        for drone in self.drones:
            drone.update(dt)
        
        # Check if formation transition is complete
        all_in_position = True
        for drone in self.drones:
            if (abs(drone.x - drone.target_x) > 0.5 or 
                abs(drone.y - drone.target_y) > 0.5 or 
                abs(drone.z - drone.target_z) > 0.5):
                all_in_position = False
                break
        
        # If all drones are in position but we haven't started timing yet
        if all_in_position and not self.formation_complete and self.formation_start_time == 0:
            self.formation_start_time = time.time()
            print(f"Formation achieved. Holding for {FORMATION_TIME} seconds...")
            
        # Check if we've maintained formation for the required time
        if self.formation_start_time > 0 and time.time() - self.formation_start_time >= FORMATION_TIME:
            if not self.formation_complete:
                self.formation_complete = True
                self.current_formation = self.target_formation
                print("Formation holding time complete!")
                self.advance_mission()
    
    def draw(self, screen):
        # Draw ground
        pygame.draw.rect(screen, GRAY, (0, self.screen_height - 50, self.screen_width, 50))
        
        # Draw drones
        for drone in self.drones:
            drone.draw(screen)
    
    def setup_mission(self, mission_steps):
        self.missions = mission_steps
        self.mission_step = 0
        self.mission_complete = False
        self.start_current_mission_step()
    
    def start_current_mission_step(self):
        if self.mission_step < len(self.missions):
            current_mission = self.missions[self.mission_step]
            
            if 'altitude' in current_mission:
                self.set_altitude(current_mission['altitude'])
            
            if 'formation' in current_mission:
                self.set_formation(current_mission['formation'])
            
            print(f"Starting mission step {self.mission_step + 1}: {current_mission['description']}")
    
    def advance_mission(self):
        self.mission_step += 1
        
        if self.mission_step < len(self.missions):
            self.start_current_mission_step()
        else:
            self.mission_complete = True
            print("Mission completed successfully!")

# Main game function
def main():
    pygame.init()
    
    # Set up the screen
    screen_width, screen_height = 800, 600
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption("İHA Sürü Formasyon Simülasyonu")
    
    clock = pygame.time.Clock()
    font = pygame.font.SysFont('Arial', 24)
    
    # Load drone image
    try:
        drone_img = pygame.image.load("drone.png").convert_alpha()
        # Resize the image to a reasonable size
        drone_img = pygame.transform.scale(drone_img, (40, 40))
    except pygame.error:
        print("Warning: Could not load drone.png. Using fallback circle shapes.")
        drone_img = None
    
    # Create formation manager
    manager = FormationManager(NUM_DRONES, screen_width, screen_height, drone_img)
    
    # Set up mission according to the scenario
    mission_steps = [
        {
            'description': "Drone'lar belirlenen yükseklige ulaşıyor",
            'altitude': 15,
            'formation': Formation.RANDOM
        },
        {
            'description': 'OK başı formasyonuna geçiliyor ve parametre süre bekleniyor...',
            'formation': Formation.ARROWHEAD
        },
        {
            'description': 'çizgi formasyonuna geçiliyor ve parametre süre bekleniyor...',
            'formation': Formation.LINE
        },
        {
            'description': 'Landing while maintaining Line formation',
            'altitude': 0
        }
    ]
    
    manager.setup_mission(mission_steps)
    
    # Main game loop
    running = True
    last_time = time.time()
    
    while running:
        current_time = time.time()
        dt = current_time - last_time
        last_time = current_time
        
        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_r:
                    # Reset simulation
                    manager = FormationManager(NUM_DRONES, screen_width, screen_height, drone_img)
                    manager.setup_mission(mission_steps)
        
        # Update
        manager.update(dt)
        
        # Draw
        screen.fill(WHITE)
        
        # Draw formation name and mission status
        formation_names = {
            Formation.RANDOM: "Random",
            Formation.LINE: "Line",
            Formation.V_SHAPE: "V-Shape",
            Formation.ARROWHEAD: "Arrowhead"
        }
        
        formation_text = f"Formation: {formation_names[manager.target_formation]}"
        formation_surface = font.render(formation_text, True, BLACK)
        screen.blit(formation_surface, (20, 20))
        
        if manager.formation_start_time > 0 and not manager.formation_complete:
            elapsed = time.time() - manager.formation_start_time
            remaining = max(0, FORMATION_TIME - elapsed)
            time_text = f"Holding: {remaining:.1f}s remaining"
            time_surface = font.render(time_text, True, BLUE)
            screen.blit(time_surface, (20, 50))
        
        if manager.mission_step < len(manager.missions):
            mission_text = f"Step {manager.mission_step + 1}: {manager.missions[manager.mission_step]['description']}"
            mission_surface = font.render(mission_text, True, BLACK)
            screen.blit(mission_surface, (20, 80))
        
        if manager.mission_complete:
            complete_text = "Mission Complete!"
            complete_surface = font.render(complete_text, True, GREEN)
            screen.blit(complete_surface, (screen_width//2 - 100, 20))
        
        # Draw instructions
        instruction_text = "Press R to restart, ESC to quit"
        instruction_surface = pygame.font.SysFont('Arial', 16).render(instruction_text, True, BLACK)
        screen.blit(instruction_surface, (screen_width - 250, screen_height - 30))
        
        # Draw drones
        manager.draw(screen)
        
        pygame.display.flip()
        clock.tick(120)
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

In [ ]:
# 5.2 Sürü Halinde Navigasyon Görevi
import pygame
import sys
import math
import time
from enum import Enum
import random

# Formation types
class Formation(Enum):
    RANDOM = 0
    LINE = 1
    V_SHAPE = 2
    ARROWHEAD = 3

# Mission states
class MissionState(Enum):
    TAKEOFF = 0
    NAVIGATE_TO_WAYPOINT = 1
    WAIT_AT_WAYPOINT = 2
    LANDING = 3
    COMPLETED = 4

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
ORANGE = (255, 165, 0)
GRAY = (200, 200, 200)

# Simulation parameters (modifiable)
NUM_DRONES = 5  # En az 3 drone
DRONE_SPACING = 50  # X metre (drone arası mesafe)
MAX_ALTITUDE = 15  # Z metre (uçuş irtifası)
WAYPOINT_ARRIVAL_TIME = 10  # T1 saniye (ara noktalara ulaşma süresi)
WAYPOINT_WAIT_TIME = 5  # T2 saniye (ara noktada bekleme süresi)

# Transition speeds
ALTITUDE_CHANGE_SPEED = 100  # Yükselme/alçalma hızı (metre/saniye)
FORMATION_TRANSITION_SPEED = 100  # Formasyon geçiş hızı (metre/saniye)
NAVIGATION_SPEED = 150.0  # Navigasyon hızı (metre/saniye) - adjusted for simulation

# Waypoint class
class Waypoint:
    def __init__(self, x, y, name=""):
        self.x = x
        self.y = y
        self.name = name
        self.size = 15
        self.color = YELLOW
        self.reached = False
    
    def draw(self, screen):
        # Draw waypoint
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), self.size)
        
        # Draw waypoint name
        if self.name:
            font = pygame.font.SysFont('Arial', 14)
            text = font.render(self.name, True, BLACK)
            screen.blit(text, (int(self.x) - 20, int(self.y) - 30))

# Drone class
class Drone:
    def __init__(self, id, x, y, z=0):
        self.id = id
        self.x = x
        self.y = y
        self.z = z  # Altitude
        self.target_x = x
        self.target_y = y
        self.target_z = z
        self.size = 20
        self.color = (random.randint(100, 200), random.randint(100, 200), random.randint(100, 200))
        self.connected = True  # Connection status to ground control
    
    def set_connection_status(self, status):
        self.connected = status
        # Change color when connection is lost
        if not self.connected:
            self.color = (200, 50, 50)  # More red when disconnected
    
    def update(self, dt):
        # Move towards target position
        dx = self.target_x - self.x
        dy = self.target_y - self.y
        dz = self.target_z - self.z
        
        if abs(dx) > 0.1:
            self.x += min(FORMATION_TRANSITION_SPEED * dt, abs(dx)) * (1 if dx > 0 else -1)
        if abs(dy) > 0.1:
            self.y += min(FORMATION_TRANSITION_SPEED * dt, abs(dy)) * (1 if dy > 0 else -1)
        if abs(dz) > 0.1:
            self.z += min(ALTITUDE_CHANGE_SPEED * dt, abs(dz)) * (1 if dz > 0 else -1)
    
    def draw(self, screen, scale_factor=10):
        # Calculate display position (altitude affects size and transparency)
        altitude_factor = 1 + (self.z / MAX_ALTITUDE * 0.5)
        display_size = int(self.size * altitude_factor)
        
        # Altitude affects transparency
        alpha = min(255, max(100, int(255 - (self.z / MAX_ALTITUDE * 100))))
        
        # Create drone surface with transparency
        drone_surface = pygame.Surface((display_size, display_size), pygame.SRCALPHA)
        
        # Draw drone body
        pygame.draw.circle(drone_surface, (*self.color, alpha), 
                         (display_size//2, display_size//2), display_size//2)
        
        # Draw drone ID
        font = pygame.font.SysFont('Arial', 10)
        id_text = font.render(str(self.id), True, BLACK)
        drone_surface.blit(id_text, (display_size//2 - 4, display_size//2 - 5))
        
        # Draw connection status
        status_color = GREEN if self.connected else RED
        pygame.draw.circle(drone_surface, status_color, (display_size-5, 5), 3)
        
        # Draw altitude indicator
        altitude_text = font.render(f"{int(self.z)}m", True, BLACK)
        screen.blit(altitude_text, (int(self.x) - 10, int(self.y) - display_size - 5))
        
        # Blit the drone to the screen
        screen.blit(drone_surface, (int(self.x) - display_size//2, int(self.y) - display_size//2))
        
        # Draw shadow on the ground
        shadow_size = max(5, int(display_size * 0.5))
        shadow_surface = pygame.Surface((shadow_size, shadow_size), pygame.SRCALPHA)
        shadow_alpha = max(50, 150 - int(self.z * 5))
        pygame.draw.ellipse(shadow_surface, (0, 0, 0, shadow_alpha), 
                         (0, 0, shadow_size, shadow_size//2))
        screen.blit(shadow_surface, (int(self.x) - shadow_size//2, int(self.y) + 20))

# Swarm Navigation Manager
class NavigationManager:
    def __init__(self, num_drones, screen_width, screen_height):
        self.screen_width = screen_width
        self.screen_height = screen_height
        self.drones = []
        self.waypoints = []
        self.current_formation = Formation.RANDOM
        self.target_formation = Formation.RANDOM
        self.formation_complete = False
        self.mission_state = MissionState.TAKEOFF
        self.mission_start_time = time.time()
        self.state_start_time = time.time()
        self.current_waypoint_index = 0
        self.total_distance_to_waypoint = 0
        self.connection_status = True
        
        # Create waypoints - will be positioned based on screen size
        self.create_waypoints()
        
        # Create drones in initial random positions on the ground
        base_y = screen_height - 100
        for i in range(num_drones):
            x = random.randint(100, screen_width - 100)
            self.drones.append(Drone(i+1, x, base_y))
        
        # Initialize formation
        self.set_formation(Formation.ARROWHEAD)  # Starting with arrowhead formation
    
    def create_waypoints(self):
        # Clear existing waypoints
        self.waypoints = []
        
        # Create waypoints at reasonable positions on the screen
        margin = 100
        start_x = margin
        start_y = self.screen_height // 2
        
        # Start position
        self.start_position = (start_x, start_y)
        
        # First waypoint
        wp1_x = self.screen_width // 3
        wp1_y = self.screen_height // 3
        self.waypoints.append(Waypoint(wp1_x, wp1_y, "Waypoint 1"))
        
        # Second waypoint
        wp2_x = self.screen_width * 2 // 3
        wp2_y = self.screen_height // 3
        self.waypoints.append(Waypoint(wp2_x, wp2_y, "Waypoint 2"))
        
        # Target (final) waypoint
        target_x = self.screen_width - margin
        target_y = self.screen_height // 2
        self.waypoints.append(Waypoint(target_x, target_y, "Target"))
        
        # Set initial position for drones near the start
        for drone in self.drones:
            drone.x = start_x
            drone.y = start_y
            drone.target_x = start_x
            drone.target_y = start_y
    
    def set_formation(self, formation_type):
        self.target_formation = formation_type
        self.formation_complete = False
        
        # Get the center position of the swarm
        if self.mission_state == MissionState.TAKEOFF:
            center_x, center_y = self.start_position
        elif self.mission_state == MissionState.NAVIGATE_TO_WAYPOINT or self.mission_state == MissionState.WAIT_AT_WAYPOINT:
            if self.current_waypoint_index < len(self.waypoints):
                waypoint = self.waypoints[self.current_waypoint_index]
                center_x, center_y = waypoint.x, waypoint.y
            else:
                center_x, center_y = self.screen_width // 2, self.screen_height // 2
        else:
            # Use current position of lead drone as center
            center_x = sum(drone.x for drone in self.drones) / len(self.drones)
            center_y = sum(drone.y for drone in self.drones) / len(self.drones)
        
        # Apply formation positions based on center
        if formation_type == Formation.LINE:
            # Line formation
            total_width = (len(self.drones) - 1) * DRONE_SPACING
            start_x = center_x - total_width // 2
            
            for i, drone in enumerate(self.drones):
                drone.target_x = start_x + i * DRONE_SPACING
                drone.target_y = center_y
                
        elif formation_type == Formation.V_SHAPE:
            # V formation
            angle = math.pi / 6  # 30 degrees
            
            for i, drone in enumerate(self.drones):
                if i == 0:  # Lead drone
                    drone.target_x = center_x
                    drone.target_y = center_y - DRONE_SPACING
                else:
                    # Alternate between left and right sides
                    side = 1 if i % 2 == 1 else -1
                    position = (i + 1) // 2
                    
                    drone.target_x = center_x + side * position * DRONE_SPACING * math.sin(angle)
                    drone.target_y = center_y + position * DRONE_SPACING * math.cos(angle)
                    
        elif formation_type == Formation.ARROWHEAD:
            # Arrowhead (Lambda Λ) formation
            angle = math.pi / 4  # 45 degrees
            
            for i, drone in enumerate(self.drones):
                if i == 0:  # Lead drone at the front
                    drone.target_x = center_x
                    drone.target_y = center_y - DRONE_SPACING
                else:
                    # Alternate between left and right sides
                    side = 1 if i % 2 == 1 else -1
                    position = (i + 1) // 2
                    
                    drone.target_x = center_x + side * position * DRONE_SPACING * math.sin(angle)
                    drone.target_y = center_y + position * DRONE_SPACING * math.cos(angle)
                    
        else:  # RANDOM or default
            # Keep current positions
            pass
    
    def set_altitude(self, altitude):
        for drone in self.drones:
            drone.target_z = altitude
    
    def move_to_waypoint(self, waypoint_index):
        if waypoint_index < len(self.waypoints):
            waypoint = self.waypoints[waypoint_index]
            
            # Calculate movement vector to waypoint for lead drone
            lead_drone = self.drones[0]
            dx = waypoint.x - lead_drone.x
            dy = waypoint.y - lead_drone.y
            distance = math.sqrt(dx*dx + dy*dy)
            self.total_distance_to_waypoint = distance
            
            # Move all drones maintaining formation
            for drone in self.drones:
                # Calculate offset from lead drone based on current formation
                offset_x = drone.target_x - lead_drone.target_x
                offset_y = drone.target_y - lead_drone.target_y
                
                # Set target as waypoint plus offset
                drone.target_x = waypoint.x + offset_x
                drone.target_y = waypoint.y + offset_y
    
    def set_connection_status(self, status):
        self.connection_status = status
        # Update all drones' connection status
        for drone in self.drones:
            drone.set_connection_status(status)
    
    def update(self, dt):
        # Update all drones
        for drone in self.drones:
            drone.update(dt)
        
        # State machine for mission progression
        if self.mission_state == MissionState.TAKEOFF:
            # Check if all drones have reached target altitude
            all_at_altitude = all(abs(drone.z - drone.target_z) < 0.5 for drone in self.drones)
            
            if all_at_altitude and time.time() - self.state_start_time >= 2:  # Add a small delay
                print("Takeoff complete. Moving to first waypoint.")
                self.mission_state = MissionState.NAVIGATE_TO_WAYPOINT
                self.state_start_time = time.time()
                self.move_to_waypoint(self.current_waypoint_index)
        
        elif self.mission_state == MissionState.NAVIGATE_TO_WAYPOINT:
            # Check if we've reached the waypoint
            lead_drone = self.drones[0]
            current_waypoint = self.waypoints[self.current_waypoint_index]
            
            dx = current_waypoint.x - lead_drone.x
            dy = current_waypoint.y - lead_drone.y
            distance_to_waypoint = math.sqrt(dx*dx + dy*dy)
            
            # Calculate how long we've been navigating
            navigation_time = time.time() - self.state_start_time
            
            # Check if we've arrived at the waypoint (or reached max time)
            if distance_to_waypoint < 10 or navigation_time >= WAYPOINT_ARRIVAL_TIME:
                print(f"Reached waypoint {self.current_waypoint_index + 1}")
                current_waypoint.reached = True
                current_waypoint.color = GREEN
                
                # Move to wait state
                self.mission_state = MissionState.WAIT_AT_WAYPOINT
                self.state_start_time = time.time()
                
                # If this is the first waypoint, simulate connection loss
                if self.current_waypoint_index == 0 and self.connection_status:
                    # Will trigger connection loss after waiting at first waypoint
                    pass
            else:
                # Continue moving towards waypoint
                # Scale the speed to ensure we reach in the allowed time
                progress = navigation_time / WAYPOINT_ARRIVAL_TIME
                remaining_time = WAYPOINT_ARRIVAL_TIME - navigation_time
                
                # Adjust speed if needed
                if remaining_time > 0:
                    required_speed = distance_to_waypoint / remaining_time
                    if required_speed > NAVIGATION_SPEED:
                        # Use the existing variable, no need for global declaration here
                        # since we're only reading it, not modifying it
                        adjusted_speed = required_speed * 1.1  # Add margin
                        # Use the adjusted speed for calculations without changing the global
        
        elif self.mission_state == MissionState.WAIT_AT_WAYPOINT:
            wait_time = time.time() - self.state_start_time
            
            # If this is the first waypoint and connection hasn't been lost yet
            if self.current_waypoint_index == 0 and self.connection_status and wait_time >= 5:
                print("Connection to ground control lost!")
                self.set_connection_status(False)
            
            # Check if we've waited long enough
            if wait_time >= WAYPOINT_WAIT_TIME:
                self.current_waypoint_index += 1
                
                # Check if there are more waypoints
                if self.current_waypoint_index < len(self.waypoints):
                    print(f"Moving to waypoint {self.current_waypoint_index + 1}")
                    self.mission_state = MissionState.NAVIGATE_TO_WAYPOINT
                    self.state_start_time = time.time()
                    self.move_to_waypoint(self.current_waypoint_index)
                else:
                    # All waypoints visited, time to land
                    print("All waypoints visited. Beginning landing procedure.")
                    self.mission_state = MissionState.LANDING
                    self.state_start_time = time.time()
                    self.set_altitude(0)  # Set target altitude to ground level
        
        elif self.mission_state == MissionState.LANDING:
            # Check if all drones have landed
            all_landed = all(drone.z < 0.5 for drone in self.drones)
            
            if all_landed:
                print("Mission completed successfully!")
                self.mission_state = MissionState.COMPLETED
    
    def draw(self, screen):
        # Draw ground
        pygame.draw.rect(screen, GRAY, (0, self.screen_height - 50, self.screen_width, 50))
        
        # Draw path between waypoints
        if self.waypoints:
            start_pos = self.start_position
            # Draw path from start to first waypoint
            pygame.draw.line(screen, GRAY, start_pos, (self.waypoints[0].x, self.waypoints[0].y), 2)
            
            # Draw paths between waypoints
            for i in range(len(self.waypoints) - 1):
                pygame.draw.line(screen, GRAY, 
                               (self.waypoints[i].x, self.waypoints[i].y),
                               (self.waypoints[i+1].x, self.waypoints[i+1].y), 2)
        
        # Draw waypoints
        for wp in self.waypoints:
            wp.draw(screen)
        
        # Draw drones
        for drone in self.drones:
            drone.draw(screen)
        
        # Draw mission information
        font = pygame.font.SysFont('Arial', 24)
        
        # Show mission state
        state_text = f"State: {self.mission_state.name}"
        state_surface = font.render(state_text, True, BLACK)
        screen.blit(state_surface, (20, 20))
        
        # Show formation type
        formation_names = {
            Formation.RANDOM: "Random",
            Formation.LINE: "Line",
            Formation.V_SHAPE: "V-Shape",
            Formation.ARROWHEAD: "Arrowhead"
        }
        formation_text = f"Formation: {formation_names[self.target_formation]}"
        formation_surface = font.render(formation_text, True, BLACK)
        screen.blit(formation_surface, (20, 50))
        
        # Show connection status
        connection_text = f"Connection: {'Connected' if self.connection_status else 'DISCONNECTED'}"
        connection_color = GREEN if self.connection_status else RED
        connection_surface = font.render(connection_text, True, connection_color)
        screen.blit(connection_surface, (20, 80))
        
        # Show current waypoint or completion status
        if self.mission_state != MissionState.COMPLETED:
            if self.mission_state == MissionState.WAIT_AT_WAYPOINT:
                wait_time = time.time() - self.state_start_time
                remaining = max(0, WAYPOINT_WAIT_TIME - wait_time)
                waypoint_text = f"Waiting at Waypoint {self.current_waypoint_index + 1}: {remaining:.1f}s remaining"
                waypoint_surface = font.render(waypoint_text, True, BLUE)
                screen.blit(waypoint_surface, (20, 110))
            elif self.mission_state == MissionState.NAVIGATE_TO_WAYPOINT:
                nav_time = time.time() - self.state_start_time
                remaining = max(0, WAYPOINT_ARRIVAL_TIME - nav_time)
                waypoint_text = f"Moving to Waypoint {self.current_waypoint_index + 1}: {remaining:.1f}s remaining"
                waypoint_surface = font.render(waypoint_text, True, BLUE)
                screen.blit(waypoint_surface, (20, 110))
        else:
            complete_text = "Mission Complete!"
            complete_surface = font.render(complete_text, True, GREEN)
            screen.blit(complete_surface, (20, 110))
        
        # Show mission time
        mission_time = time.time() - self.mission_start_time
        time_text = f"Mission Time: {mission_time:.1f}s"
        time_surface = font.render(time_text, True, BLACK)
        screen.blit(time_surface, (self.screen_width - 250, 20))
        
        # Display instructions
        instruction_font = pygame.font.SysFont('Arial', 16)
        instructions = [
            "Controls:",
            "R - Restart simulation",
            "C - Toggle ground control connection",
            "1-4 - Change formation (1:Random, 2:Line, 3:V-Shape, 4:Arrowhead)",
            "ESC - Quit"
        ]
        
        for i, instruction in enumerate(instructions):
            inst_surface = instruction_font.render(instruction, True, BLACK)
            screen.blit(inst_surface, (self.screen_width - 350, self.screen_height - 100 + i*20))

# Main game function
def main():
    pygame.init()
    
    # Set up the screen
    screen_width, screen_height = 1000, 700
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption("İHA Sürü Navigasyon Simülasyonu")
    
    clock = pygame.time.Clock()
    
    # Create navigation manager
    manager = NavigationManager(NUM_DRONES, screen_width, screen_height)
    
    # Initial mission setup
    manager.set_altitude(MAX_ALTITUDE)
    
    # Main game loop
    running = True
    last_time = time.time()
    
    while running:
        current_time = time.time()
        dt = current_time - last_time
        last_time = current_time
        
        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_r:
                    # Reset simulation
                    manager = NavigationManager(NUM_DRONES, screen_width, screen_height)
                    manager.set_altitude(MAX_ALTITUDE)
                elif event.key == pygame.K_c:
                    # Toggle connection status
                    manager.set_connection_status(not manager.connection_status)
                elif event.key == pygame.K_1:
                    manager.set_formation(Formation.RANDOM)
                elif event.key == pygame.K_2:
                    manager.set_formation(Formation.LINE)
                elif event.key == pygame.K_3:
                    manager.set_formation(Formation.V_SHAPE)
                elif event.key == pygame.K_4:
                    manager.set_formation(Formation.ARROWHEAD)
        
        # Update
        manager.update(dt)
        
        # Draw
        screen.fill(WHITE)
        manager.draw(screen)
        
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

In [ ]:
# 5.3 Sürüden Birey Çıkarma/Ekleme Görevi

import pygame
import sys
import math
import time
import random
from enum import Enum

# Formation types
class Formation(Enum):
    RANDOM = 0
    LINE = 1
    V_SHAPE = 2
    ARROWHEAD = 3

# Mission states
class MissionState(Enum):
    TAKEOFF = 0
    HOLD_FORMATION = 1
    NAVIGATE_TO_WAYPOINT = 2
    WAIT_AT_WAYPOINT = 3
    DRONE_LEAVING = 4
    DRONE_JOINING = 5
    FINAL_WAIT = 6
    LANDING = 7
    COMPLETED = 8

# Drone states
class DroneState(Enum):
    GROUNDED = 0
    ACTIVE = 1
    RETURNING = 2
    WAITING = 3
    JOINING = 4

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
ORANGE = (255, 165, 0)
PURPLE = (128, 0, 128)
GRAY = (200, 200, 200)

# Simulation parameters (modifiable)
NUM_DRONES = 5  # En az 3 drone
DRONE_SPACING = 50  # X metre (drone arası mesafe)
MAX_ALTITUDE = 15  # Z metre (uçuş irtifası)
FORMATION_TIME = 10  # T saniye (formasyon koruma süresi)
WAYPOINT_WAIT_TIME = 15  # Saniye (ara noktada bekleme süresi)
TRANSITION_TIME = 10  # Saniye (geçiş için maksimum süre)

# Transition speeds
ALTITUDE_CHANGE_SPEED = 100.0  # Yükselme/alçalma hızı (metre/saniye)
FORMATION_TRANSITION_SPEED = 100  # Formasyon geçiş hızı (metre/saniye)
NAVIGATION_SPEED = 150.0  # Navigasyon hızı (metre/saniye)

# Waypoint class
class Waypoint:
    def __init__(self, x, y, name=""):
        self.x = x
        self.y = y
        self.name = name
        self.size = 15
        self.color = YELLOW
        self.reached = False
    
    def draw(self, screen):
        # Draw waypoint
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), self.size)
        
        # Draw waypoint name
        if self.name:
            font = pygame.font.SysFont('Arial', 14)
            text = font.render(self.name, True, BLACK)
            screen.blit(text, (int(self.x) - 20, int(self.y) - 30))

# Drone class
class Drone:
    def __init__(self, id, x, y, z=0, state=DroneState.GROUNDED):
        self.id = id
        self.x = x
        self.y = y
        self.z = z  # Altitude
        self.target_x = x
        self.target_y = y
        self.target_z = z
        self.size = 20
        self.state = state
        self.formation_position = None  # To store position in formation
        
        # Assign color based on state
        self.update_color()
    
    def update_color(self):
        if self.state == DroneState.GROUNDED:
            self.color = GRAY
        elif self.state == DroneState.ACTIVE:
            self.color = BLUE
        elif self.state == DroneState.RETURNING:
            self.color = RED
        elif self.state == DroneState.WAITING:
            self.color = ORANGE
        elif self.state == DroneState.JOINING:
            self.color = GREEN
    
    def set_state(self, new_state):
        self.state = new_state
        self.update_color()
    
    def update(self, dt):
        # Move towards target position
        dx = self.target_x - self.x
        dy = self.target_y - self.y
        dz = self.target_z - self.z
        
        if abs(dx) > 0.1:
            self.x += min(FORMATION_TRANSITION_SPEED * dt, abs(dx)) * (1 if dx > 0 else -1)
        if abs(dy) > 0.1:
            self.y += min(FORMATION_TRANSITION_SPEED * dt, abs(dy)) * (1 if dy > 0 else -1)
        if abs(dz) > 0.1:
            self.z += min(ALTITUDE_CHANGE_SPEED * dt, abs(dz)) * (1 if dz > 0 else -1)
    
    def draw(self, screen, scale_factor=10):
        # Calculate display position (altitude affects size and transparency)
        altitude_factor = 1 + (self.z / MAX_ALTITUDE * 0.5)
        display_size = int(self.size * altitude_factor)
        
        # Altitude affects transparency
        alpha = min(255, max(100, int(255 - (self.z / MAX_ALTITUDE * 100))))
        
        # Create drone surface with transparency
        drone_surface = pygame.Surface((display_size, display_size), pygame.SRCALPHA)
        
        # Draw drone body
        pygame.draw.circle(drone_surface, (*self.color, alpha), 
                         (display_size//2, display_size//2), display_size//2)
        
        # Draw drone ID
        font = pygame.font.SysFont('Arial', 10)
        id_text = font.render(str(self.id), True, BLACK)
        drone_surface.blit(id_text, (display_size//2 - 4, display_size//2 - 5))
        
        # Draw state indicator
        state_text = font.render(self.state.name[0], True, BLACK)
        drone_surface.blit(state_text, (display_size//2 - 4, display_size//2 + 5))
        
        # Draw altitude indicator
        altitude_text = font.render(f"{int(self.z)}m", True, BLACK)
        screen.blit(altitude_text, (int(self.x) - 10, int(self.y) - display_size - 5))
        
        # Blit the drone to the screen
        screen.blit(drone_surface, (int(self.x) - display_size//2, int(self.y) - display_size//2))
        
        # Draw shadow on the ground
        shadow_size = max(5, int(display_size * 0.5))
        shadow_surface = pygame.Surface((shadow_size, shadow_size), pygame.SRCALPHA)
        shadow_alpha = max(50, 150 - int(self.z * 5))
        pygame.draw.ellipse(shadow_surface, (0, 0, 0, shadow_alpha), 
                         (0, 0, shadow_size, shadow_size//2))
        screen.blit(shadow_surface, (int(self.x) - shadow_size//2, int(self.y) + 20))

# Swarm Replacement Manager
class ReplacementManager:
    def __init__(self, num_drones, screen_width, screen_height):
        self.screen_width = screen_width
        self.screen_height = screen_height
        self.drones = []
        self.waypoints = []
        self.home_position = (screen_width - 100, screen_height - 100)  # Home location for returning drones
        self.current_formation = Formation.ARROWHEAD
        self.formation_complete = False
        self.mission_state = MissionState.TAKEOFF
        self.mission_start_time = time.time()
        self.state_start_time = time.time()
        self.leaving_drone = None
        self.joining_drone = None
        self.joining_drone_target_position = None
        
        # Create waypoints
        self.create_waypoints()
        
        # Create drones - some active, some waiting on ground
        base_y = screen_height - 100
        
        # Create active drones
        for i in range(num_drones - 1):  # Leave one drone on ground
            x = 100 + i * 50  # Spread them out a bit
            drone = Drone(i+1, x, base_y)
            self.drones.append(drone)
        
        # Create one waiting drone
        waiting_drone = Drone(num_drones, 200, base_y, state=DroneState.WAITING)
        waiting_drone.target_z = 0  # Make sure it stays on the ground
        self.drones.append(waiting_drone)
        self.joining_drone = waiting_drone
    
    def create_waypoints(self):
        # Clear existing waypoints
        self.waypoints = []
        
        # Create waypoint at reasonable position on the screen
        waypoint_x = self.screen_width // 2
        waypoint_y = self.screen_height // 3
        self.waypoints.append(Waypoint(waypoint_x, waypoint_y, "Waypoint"))
    
    def set_formation(self, formation_type, center_x=None, center_y=None):
        self.current_formation = formation_type
        
        # If no center provided, use current average position of active drones
        if center_x is None or center_y is None:
            active_drones = [drone for drone in self.drones if drone.state == DroneState.ACTIVE]
            if active_drones:
                center_x = sum(drone.x for drone in active_drones) / len(active_drones)
                center_y = sum(drone.y for drone in active_drones) / len(active_drones)
            else:
                center_x = self.screen_width // 2
                center_y = self.screen_height // 2
        
        # Calculate positions based on formation
        if formation_type == Formation.LINE:
            active_drones = [drone for drone in self.drones if drone.state == DroneState.ACTIVE]
            total_width = (len(active_drones) - 1) * DRONE_SPACING
            start_x = center_x - total_width // 2
            
            for i, drone in enumerate(active_drones):
                # Store formation position
                formation_pos = (start_x + i * DRONE_SPACING, center_y)
                drone.formation_position = formation_pos
                
                # Only set target for active drones
                if drone.state == DroneState.ACTIVE:
                    drone.target_x, drone.target_y = formation_pos
                
        elif formation_type == Formation.V_SHAPE:
            active_drones = [drone for drone in self.drones if drone.state == DroneState.ACTIVE]
            angle = math.pi / 6  # 30 degrees
            
            for i, drone in enumerate(active_drones):
                if i == 0:  # Lead drone
                    formation_pos = (center_x, center_y - DRONE_SPACING)
                else:
                    # Alternate between left and right sides
                    side = 1 if i % 2 == 1 else -1
                    position = (i + 1) // 2
                    
                    formation_pos = (
                        center_x + side * position * DRONE_SPACING * math.sin(angle),
                        center_y + position * DRONE_SPACING * math.cos(angle)
                    )
                
                drone.formation_position = formation_pos
                
                # Only set target for active drones
                if drone.state == DroneState.ACTIVE:
                    drone.target_x, drone.target_y = formation_pos
                    
        elif formation_type == Formation.ARROWHEAD:
            active_drones = [drone for drone in self.drones if drone.state == DroneState.ACTIVE]
            angle = math.pi / 4  # 45 degrees
            
            for i, drone in enumerate(active_drones):
                if i == 0:  # Lead drone at the front
                    formation_pos = (center_x, center_y - DRONE_SPACING)
                else:
                    # Alternate between left and right sides
                    side = 1 if i % 2 == 1 else -1
                    position = (i + 1) // 2
                    
                    formation_pos = (
                        center_x + side * position * DRONE_SPACING * math.sin(angle),
                        center_y + position * DRONE_SPACING * math.cos(angle)
                    )
                
                drone.formation_position = formation_pos
                
                # Only set target for active drones
                if drone.state == DroneState.ACTIVE:
                    drone.target_x, drone.target_y = formation_pos
    
    def set_altitude(self, altitude, drone_states=None):
        # If no specific drone states provided, apply to all active drones
        if drone_states is None:
            drone_states = [DroneState.ACTIVE]
            
        for drone in self.drones:
            if drone.state in drone_states:
                drone.target_z = altitude
    
    def move_to_waypoint(self, waypoint_index):
        if waypoint_index < len(self.waypoints):
            waypoint = self.waypoints[waypoint_index]
            
            # Get active drones
            active_drones = [drone for drone in self.drones if drone.state == DroneState.ACTIVE]
            
            if active_drones:
                # Calculate center of formation
                center_x = waypoint.x
                center_y = waypoint.y
                
                # Reset formation with waypoint as center
                self.set_formation(self.current_formation, center_x, center_y)
    
    def select_drone_to_leave(self):
        # Get list of active drones
        active_drones = [drone for drone in self.drones if drone.state == DroneState.ACTIVE]
        
        if active_drones:
            # Select a random drone to leave (not the lead drone)
            if len(active_drones) > 1:
                self.leaving_drone = random.choice(active_drones[1:])
            else:
                self.leaving_drone = active_drones[0]
                
            # Set its state to returning
            self.leaving_drone.set_state(DroneState.RETURNING)
            
            # Set target to home position
            self.leaving_drone.target_x, self.leaving_drone.target_y = self.home_position
            self.leaving_drone.target_z = 0  # Land
            
            # Save the position it's leaving from for joining drone
            self.joining_drone_target_position = self.leaving_drone.formation_position
            
            print(f"Drone {self.leaving_drone.id} is leaving the swarm")
            
            # Recalculate formation for remaining drones
            self.set_formation(self.current_formation)
    
    def prepare_drone_to_join(self):
        # Find the waiting drone
        for drone in self.drones:
            if drone.state == DroneState.WAITING:
                self.joining_drone = drone
                drone.set_state(DroneState.JOINING)
                
                # Set initial rise to formation altitude
                drone.target_z = MAX_ALTITUDE
                
                print(f"Drone {drone.id} is preparing to join the swarm")
                break
    
    def complete_drone_joining(self):
        if self.joining_drone and self.joining_drone.state == DroneState.JOINING:
            # Set to active state
            self.joining_drone.set_state(DroneState.ACTIVE)
            
            # Add to formation at the position left by the leaving drone
            if self.joining_drone_target_position:
                self.joining_drone.target_x, self.joining_drone.target_y = self.joining_drone_target_position
            
            print(f"Drone {self.joining_drone.id} has joined the swarm")
            
            # Recalculate formation
            self.set_formation(self.current_formation)
    
    def all_drones_in_position(self):
        for drone in self.drones:
            # Skip non-active drones for position check
            if drone.state not in [DroneState.ACTIVE, DroneState.JOINING, DroneState.RETURNING]:
                continue
                
            # Check if drone is at its target position
            if (abs(drone.x - drone.target_x) > 0.5 or 
                abs(drone.y - drone.target_y) > 0.5 or 
                abs(drone.z - drone.target_z) > 0.5):
                return False
        return True
    
    def update(self, dt):
        # Update all drones
        for drone in self.drones:
            drone.update(dt)
        
        # State machine for mission progression
        current_time = time.time()
        state_duration = current_time - self.state_start_time
        
        if self.mission_state == MissionState.TAKEOFF:
            # Set all initially active drones to take off
            if state_duration < 0.1:  # Just entered state
                active_drones = [drone for drone in self.drones if drone.state != DroneState.WAITING]
                for drone in active_drones:
                    drone.set_state(DroneState.ACTIVE)
                    drone.target_z = MAX_ALTITUDE
                
                # Set initial formation
                self.set_formation(Formation.ARROWHEAD)
                
            # Check if all active drones have reached target altitude
            if self.all_drones_in_position() and state_duration >= 2:
                print("Takeoff complete. Holding formation.")
                self.mission_state = MissionState.HOLD_FORMATION
                self.state_start_time = current_time
        
        elif self.mission_state == MissionState.HOLD_FORMATION:
            # Hold formation for FORMATION_TIME seconds
            if state_duration >= FORMATION_TIME:
                print("Formation holding time complete. Moving to waypoint.")
                self.mission_state = MissionState.NAVIGATE_TO_WAYPOINT
                self.state_start_time = current_time
                self.move_to_waypoint(0)  # Move to the first waypoint
        
        elif self.mission_state == MissionState.NAVIGATE_TO_WAYPOINT:
            # Check if we've arrived at the waypoint or max time reached
            if self.all_drones_in_position() or state_duration >= TRANSITION_TIME:
                print(f"Reached waypoint. Waiting for {WAYPOINT_WAIT_TIME} seconds.")
                self.waypoints[0].reached = True
                self.waypoints[0].color = GREEN
                self.mission_state = MissionState.WAIT_AT_WAYPOINT
                self.state_start_time = current_time
        
        elif self.mission_state == MissionState.WAIT_AT_WAYPOINT:
            # Wait at waypoint for WAYPOINT_WAIT_TIME seconds
            if state_duration >= WAYPOINT_WAIT_TIME:
                print("Waypoint wait time complete. Selecting drone to leave.")
                self.mission_state = MissionState.DRONE_LEAVING
                self.state_start_time = current_time
                self.select_drone_to_leave()
        
        elif self.mission_state == MissionState.DRONE_LEAVING:
            # Wait for drone to leave and reach home position
            drone_returned = (
                self.leaving_drone and 
                abs(self.leaving_drone.x - self.home_position[0]) < 5 and 
                abs(self.leaving_drone.y - self.home_position[1]) < 5 and
                self.leaving_drone.z < 1
            )
            
            # Check if drone has returned or max time reached
            if drone_returned or state_duration >= TRANSITION_TIME:
                if self.leaving_drone:
                    self.leaving_drone.set_state(DroneState.GROUNDED)
                    print(f"Drone {self.leaving_drone.id} has returned home.")
                
                print("Preparing drone to join the swarm.")
                self.mission_state = MissionState.DRONE_JOINING
                self.state_start_time = current_time
                self.prepare_drone_to_join()
        
        elif self.mission_state == MissionState.DRONE_JOINING:
            # Check if joining drone has reached altitude
            joining_at_altitude = (
                self.joining_drone and 
                abs(self.joining_drone.z - MAX_ALTITUDE) < 1
            )
            
            if joining_at_altitude and state_duration >= 2:
                # Move drone to formation position
                self.complete_drone_joining()
                
            # Check if drone has joined formation or max time reached
            drone_joined = self.joining_drone and self.joining_drone.state == DroneState.ACTIVE
            
            if (drone_joined and self.all_drones_in_position()) or state_duration >= TRANSITION_TIME:
                print("Drone has joined the swarm. Holding final formation.")
                self.mission_state = MissionState.FINAL_WAIT
                self.state_start_time = current_time
        
        elif self.mission_state == MissionState.FINAL_WAIT:
            # Hold formation for FORMATION_TIME seconds
            if state_duration >= FORMATION_TIME:
                print("Final formation holding time complete. Beginning landing.")
                self.mission_state = MissionState.LANDING
                self.state_start_time = current_time
                self.set_altitude(0, [DroneState.ACTIVE])  # Set all active drones to land
        
        elif self.mission_state == MissionState.LANDING:
            # Check if all drones have landed
            all_landed = all(drone.z < 0.5 for drone in self.drones if drone.state == DroneState.ACTIVE)
            
            # Land after max time if not all landed yet
            if all_landed or state_duration >= TRANSITION_TIME:
                print("Mission completed successfully!")
                self.mission_state = MissionState.COMPLETED
                
                # Ensure all drones are on the ground
                for drone in self.drones:
                    if drone.state == DroneState.ACTIVE:
                        drone.z = 0
                        drone.set_state(DroneState.GROUNDED)
    
    def draw(self, screen):
        # Draw ground
        pygame.draw.rect(screen, GRAY, (0, self.screen_height - 50, self.screen_width, 50))
        
        # Draw home position
        pygame.draw.circle(screen, RED, self.home_position, 20, 2)
        home_text = pygame.font.SysFont('Arial', 14).render("HOME", True, RED)
        screen.blit(home_text, (self.home_position[0] - 20, self.home_position[1] - 35))
        
        # Draw waypoints
        for wp in self.waypoints:
            wp.draw(screen)
        
        # Draw drones
        for drone in self.drones:
            drone.draw(screen)
        
        # Draw mission information
        font = pygame.font.SysFont('Arial', 24)
        
        # Show mission state
        state_text = f"State: {self.mission_state.name}"
        state_surface = font.render(state_text, True, BLACK)
        screen.blit(state_surface, (20, 20))
        
        # Show formation type
        formation_names = {
            Formation.RANDOM: "Random",
            Formation.LINE: "Line",
            Formation.V_SHAPE: "V-Shape", 
            Formation.ARROWHEAD: "Arrowhead"
        }
        formation_text = f"Formasyon: {formation_names[self.current_formation]}"
        formation_surface = font.render(formation_text, True, BLACK)
        screen.blit(formation_surface, (20, 50))
        
        # Show active drone count
        active_count = sum(1 for drone in self.drones if drone.state == DroneState.ACTIVE)
        count_text = f"Active Drones: {active_count}"
        count_surface = font.render(count_text, True, BLACK)
        screen.blit(count_surface, (20, 80))
        
        # Show current state duration
        state_time = time.time() - self.state_start_time
        time_text = f"State Time: {state_time:.1f}s"
        time_surface = font.render(time_text, True, BLACK)
        screen.blit(time_surface, (20, 110))
        
        # Show special state messages
        if self.mission_state == MissionState.DRONE_LEAVING and self.leaving_drone:
            message = f"Drone {self.leaving_drone.id} returning to home"
            message_surface = font.render(message, True, RED)
            screen.blit(message_surface, (20, 140))
        elif self.mission_state == MissionState.DRONE_JOINING and self.joining_drone:
            message = f"Drone {self.joining_drone.id} joining swarm"
            message_surface = font.render(message, True, GREEN)
            screen.blit(message_surface, (20, 140))
        
        # Show mission time
        mission_time = time.time() - self.mission_start_time
        mission_text = f"Mission Time: {mission_time:.1f}s"
        mission_surface = font.render(mission_text, True, BLACK)
        screen.blit(mission_surface, (self.screen_width - 250, 20))
        
        # Show legend
        legend_font = pygame.font.SysFont('Arial', 16)
        legend_items = [
            ("ACTIVE", BLUE),
            ("WAITING", ORANGE),
            ("JOINING", GREEN),
            ("RETURNING", RED),
            ("GROUNDED", GRAY)
        ]
        
        for i, (state, color) in enumerate(legend_items):
            # Draw color square
            pygame.draw.rect(screen, color, (self.screen_width - 250, 50 + i*25, 15, 15))
            # Draw state name
            state_text = legend_font.render(state, True, BLACK)
            screen.blit(state_text, (self.screen_width - 230, 50 + i*25))
        
        # Display instructions
        instruction_font = pygame.font.SysFont('Arial', 16)
        instructions = [
            "Controls:",
            "R - Restart simulation",
            "1-4 - Change formation (1:Random, 2:Line, 3:V-Shape, 4:Arrowhead)",
            "ESC - Quit"
        ]
        
        for i, instruction in enumerate(instructions):
            inst_surface = instruction_font.render(instruction, True, BLACK)
            screen.blit(inst_surface, (self.screen_width - 350, self.screen_height - 100 + i*20))

# Main game function
def main():
    pygame.init()
    
    # Set up the screen
    screen_width, screen_height = 1000, 700
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption("İHA Sürü Üye Değişimi Simülasyonu")
    
    clock = pygame.time.Clock()
    
    # Create swarm manager
    manager = ReplacementManager(NUM_DRONES, screen_width, screen_height)
    
    # Main game loop
    running = True
    last_time = time.time()
    
    while running:
        current_time = time.time()
        dt = current_time - last_time
        last_time = current_time
        
        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_r:
                    # Reset simulation
                    manager = ReplacementManager(NUM_DRONES, screen_width, screen_height)
                elif event.key == pygame.K_1:
                    manager.set_formation(Formation.RANDOM)
                elif event.key == pygame.K_2:
                    manager.set_formation(Formation.LINE)
                elif event.key == pygame.K_3:
                    manager.set_formation(Formation.V_SHAPE)
                elif event.key == pygame.K_4:
                    manager.set_formation(Formation.ARROWHEAD)
        
        # Update
        manager.update(dt)
        
        # Draw
        screen.fill(WHITE)
        manager.draw(screen)
        
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

In [ ]:
#5.4 Keşif Graph path 
import pygame
import random
import math
import heapq
from collections import deque
from pygame.locals import *

# Constants
WIDTH, HEIGHT = 800, 600
GRID_SIZE = 20
COLS, ROWS = WIDTH // GRID_SIZE, HEIGHT // GRID_SIZE
FPS = 30

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (200, 200, 200)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
DRONE_COLORS = [(0, 0, 255), (0, 150, 255), (100, 100, 255)]
LIGHT_BLUE = (173, 216, 230)
DARK_GRAY = (50, 50, 50)

# Drone properties
DRONE_RADIUS = 8
EXPLORATION_RADIUS = 3  # Grid cells
DRONE_SPEED = 2  # Grid cells per second
DRONE_SCAN_FREQ = 1  # Seconds between scans

class Node:
    def __init__(self, row, col, walkable=True):
        self.row = row
        self.col = col
        self.walkable = walkable
        self.visited = False
        self.path = False
        self.visited_by = set()  # Set of drone IDs that have visited this node
        self.g_score = float('inf')
        self.f_score = float('inf')
        self.parent = None
    
    def get_pos(self):
        return (self.row, self.col)
    
    def reset_path(self):
        self.path = False
        self.g_score = float('inf')
        self.f_score = float('inf')
        self.parent = None

class Grid:
    def __init__(self, cols, rows):
        self.cols = cols
        self.rows = rows
        self.grid = [[Node(row, col) for col in range(cols)] for row in range(rows)]
        self.obstacles = set()
    
    def get_node(self, row, col):
        if 0 <= row < self.rows and 0 <= col < self.cols:
            return self.grid[row][col]
        return None
    
    def get_neighbors(self, node):
        neighbors = []
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]:  # Include diagonals
            r, c = node.row + dr, node.col + dc
            if 0 <= r < self.rows and 0 <= c < self.cols and self.grid[r][c].walkable:
                neighbors.append(self.grid[r][c])
        return neighbors
    
    def reset_paths(self):
        for row in self.grid:
            for node in row:
                node.reset_path()
    
    def add_random_obstacles(self, obstacle_ratio=0.2):
        num_obstacles = int(self.rows * self.cols * obstacle_ratio)
        
        # Clear existing obstacles
        for row in self.grid:
            for node in row:
                node.walkable = True
        self.obstacles.clear()
        
        # Add new obstacles
        for _ in range(num_obstacles):
            row = random.randint(0, self.rows - 1)
            col = random.randint(0, self.cols - 1)
            node = self.get_node(row, col)
            
            # Don't create obstacles at the edges
            if row > 1 and row < self.rows - 2 and col > 1 and col < self.cols - 2:
                node.walkable = False
                self.obstacles.add((row, col))

class Drone:
    def __init__(self, grid, id, row, col):
        self.grid = grid
        self.id = id
        self.row = row
        self.col = col
        self.target_row = row
        self.target_col = col
        self.color = DRONE_COLORS[id % len(DRONE_COLORS)]
        self.path = []
        self.scan_timer = 0
        self.seen_cells = set()  # Cells this drone has seen
        self.target_found = False
        self.target_location = None
        self.move_fraction = 0.0
        self.direction = (0, 0)
        self.explored_nodes = set()
    
    def update(self, dt, global_visited, target_pos, shared_target_found):
        self.scan_timer += dt

        # If we don't have a path or we're at the target position, find a new path
        if (not self.path or (self.row == self.target_row and self.col == self.target_col)) and not self.target_found:
            if shared_target_found:
                # If another drone found the target, set that as our goal
                self.path = self.find_path(target_pos[0], target_pos[1])
                self.target_found = True
                self.target_location = target_pos
            else:
                # Otherwise explore
                self.explore(global_visited)

        # Move along path if we have one
        if self.path:
            self.move_along_path(dt)
        
        # Scan surroundings periodically
        if self.scan_timer >= DRONE_SCAN_FREQ:
            self.scan()
            self.scan_timer = 0
        
        # Check if we found target
        if not self.target_found and target_pos:
            distance = math.sqrt((self.row - target_pos[0])**2 + (self.col - target_pos[1])**2)
            if distance <= EXPLORATION_RADIUS:
                self.target_found = True
                self.target_location = target_pos
                self.path = self.find_path(target_pos[0], target_pos[1])
        
        # Update cells we've seen
        current_node = self.grid.get_node(self.row, self.col)
        if current_node:
            # Mark this node as visited by this drone
            current_node.visited_by.add(self.id)
            self.explored_nodes.add((self.row, self.col))

    def scan(self):
        # Scan cells within exploration radius
        for r in range(self.row - EXPLORATION_RADIUS, self.row + EXPLORATION_RADIUS + 1):
            for c in range(self.col - EXPLORATION_RADIUS, self.col + EXPLORATION_RADIUS + 1):
                # Check if cell is within grid bounds
                if 0 <= r < self.grid.rows and 0 <= c < self.grid.cols:
                    # Check if within circular exploration radius
                    distance = math.sqrt((r - self.row)**2 + (c - self.col)**2)
                    if distance <= EXPLORATION_RADIUS:
                        self.seen_cells.add((r, c))
    
    def explore(self, global_visited):
        # Use BFS to find the nearest unexplored cell
        frontier = deque([(self.row, self.col)])
        came_from = {(self.row, self.col): None}
        
        target_cell = None
        
        while frontier and not target_cell:
            current = frontier.popleft()
            current_row, current_col = current
            
            # Check if this cell is unexplored globally and can be reached
            if current not in global_visited and self.grid.get_node(current_row, current_col).walkable:
                target_cell = current
                break
            
            # Add neighbors to frontier
            for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]:
                neighbor_row, neighbor_col = current_row + dr, current_col + dc
                neighbor = (neighbor_row, neighbor_col)
                
                # Check if neighbor is in grid and not visited in BFS yet
                if (0 <= neighbor_row < self.grid.rows and 
                    0 <= neighbor_col < self.grid.cols and 
                    neighbor not in came_from and
                    self.grid.get_node(neighbor_row, neighbor_col).walkable):
                    frontier.append(neighbor)
                    came_from[neighbor] = current
        
        # If we found an unexplored cell, reconstruct path to it
        if target_cell:
            # Reconstruct path
            path = []
            current = target_cell
            while current != (self.row, self.col):
                path.append(current)
                current = came_from[current]
            path.reverse()
            
            # Set as target
            self.target_row, self.target_col = target_cell
            self.path = path
        else:
            # If no unexplored cells, just wander
            self.wander()
    
    def wander(self):
        # Pick a random walkable cell as a target
        walkable_cells = []
        for r in range(self.grid.rows):
            for c in range(self.grid.cols):
                node = self.grid.get_node(r, c)
                if node.walkable:
                    walkable_cells.append((r, c))
        
        if walkable_cells:
            target = random.choice(walkable_cells)
            self.target_row, self.target_col = target
            self.path = self.find_path(target[0], target[1])
    
    def find_path(self, target_row, target_col):
        # A* pathfinding
        start_node = self.grid.get_node(self.row, self.col)
        end_node = self.grid.get_node(target_row, target_col)
        
        if not start_node or not end_node or not end_node.walkable:
            return []
        
        # Reset previous path calculations
        self.grid.reset_paths()
        
        open_set = []
        closed_set = set()
        
        start_node.g_score = 0
        start_node.f_score = self.heuristic(start_node, end_node)
        heapq.heappush(open_set, (start_node.f_score, id(start_node), start_node))
        
        while open_set:
            _, _, current_node = heapq.heappop(open_set)
            
            if current_node == end_node:
                # Reconstruct path
                path = []
                while current_node != start_node:
                    path.append((current_node.row, current_node.col))
                    current_node = current_node.parent
                path.reverse()
                return path
            
            closed_set.add(current_node)
            
            for neighbor in self.grid.get_neighbors(current_node):
                if neighbor in closed_set:
                    continue
                
                # Calculate g_score (path distance from start to this neighbor through current)
                tentative_g_score = current_node.g_score + self.distance(current_node, neighbor)
                
                # If this path is better than previous ones
                if tentative_g_score < neighbor.g_score:
                    neighbor.parent = current_node
                    neighbor.g_score = tentative_g_score
                    neighbor.f_score = tentative_g_score + self.heuristic(neighbor, end_node)
                    neighbor.path = True
                    
                    # Add to open set if not already there
                    if all(n != neighbor for _, _, n in open_set):
                        heapq.heappush(open_set, (neighbor.f_score, id(neighbor), neighbor))
        
        # No path found
        return []
    
    def heuristic(self, node, target):
        # Manhattan distance
        return abs(node.row - target.row) + abs(node.col - target.col)
    
    def distance(self, node1, node2):
        # Euclidean distance for A*
        return math.sqrt((node1.row - node2.row)**2 + (node1.col - node2.col)**2)
    
    def move_along_path(self, dt):
        if not self.path:
            return
        
        if self.move_fraction == 0:
            # Starting a new movement segment
            next_pos = self.path[0]
            self.direction = (next_pos[0] - self.row, next_pos[1] - self.col)
        
        # Update movement progress
        move_speed = DRONE_SPEED * dt
        self.move_fraction += move_speed
        
        if self.move_fraction >= 1.0:
            # Completed this segment
            self.row, self.col = self.path.pop(0)
            self.move_fraction = 0.0
            self.direction = (0, 0)
        
    def draw(self, screen, cell_size):
        # Calculate interpolated position based on move_fraction
        display_row = self.row
        display_col = self.col
        
        if self.move_fraction > 0 and self.path:
            next_row, next_col = self.path[0]
            display_row = self.row + (next_row - self.row) * self.move_fraction
            display_col = self.col + (next_col - self.col) * self.move_fraction
        
        # Convert grid coordinates to pixel coordinates (center of cell)
        x = int((display_col + 0.5) * cell_size)
        y = int((display_row + 0.5) * cell_size)
        
        # Draw the drone
        pygame.draw.circle(screen, self.color, (x, y), DRONE_RADIUS)
        
        # Draw drone ID
        font = pygame.font.SysFont(None, 20)
        id_text = font.render(str(self.id), True, WHITE)
        text_rect = id_text.get_rect(center=(x, y))
        screen.blit(id_text, text_rect)
        
        # Draw exploration radius
        radius_px = EXPLORATION_RADIUS * cell_size
        pygame.draw.circle(screen, self.color, (x, y), radius_px, 1)
        
        # Draw path
        if self.path and len(self.path) > 0:
            path_points = [(int((self.col + 0.5) * cell_size), int((self.row + 0.5) * cell_size))]
            for row, col in self.path:
                path_points.append((int((col + 0.5) * cell_size), int((row + 0.5) * cell_size)))
            
            pygame.draw.lines(screen, self.color, False, path_points, 2)

class Game:
    def __init__(self):
        pygame.init()
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Drone Search Simulation")
        self.clock = pygame.time.Clock()
        self.running = True
        
        self.grid = Grid(COLS, ROWS)
        self.grid.add_random_obstacles()
        
        # Place drones at random positions
        self.drones = []
        drone_positions = set()
        for i in range(3):
            while True:
                row = random.randint(0, ROWS - 1)
                col = random.randint(0, COLS - 1)
                if (row, col) not in drone_positions and (row, col) not in self.grid.obstacles:
                    drone_positions.add((row, col))
                    self.drones.append(Drone(self.grid, i, row, col))
                    break
        
        # Place target randomly
        while True:
            self.target_row = random.randint(0, ROWS - 1)
            self.target_col = random.randint(0, COLS - 1)
            if (self.target_row, self.target_col) not in drone_positions and (self.target_row, self.target_col) not in self.grid.obstacles:
                break
        
        self.target_found = False
        self.font = pygame.font.SysFont(None, 24)
        self.simulation_time = 0
    
    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:
                    # Reset simulation
                    self.__init__()
    
    def update(self, dt):
        self.simulation_time += dt
        
        # Gather global visited cells (combine all drones' seen cells)
        global_visited = set()
        for drone in self.drones:
            global_visited.update(drone.seen_cells)
        
        # Update all drones
        target_pos = (self.target_row, self.target_col) if self.target_found else None
        
        # Check if any drone found the target
        self.target_found = any(drone.target_found for drone in self.drones)
        
        for drone in self.drones:
            drone.update(dt, global_visited, (self.target_row, self.target_col), self.target_found)
            
            # If this drone found the target, share the information
            if drone.target_found and not self.target_found:
                self.target_found = True
    
    def draw(self):
        self.screen.fill(BLACK)
        
        # Draw grid
        for row in range(ROWS):
            for col in range(COLS):
                node = self.grid.get_node(row, col)
                x = col * GRID_SIZE
                y = row * GRID_SIZE
                
                # Calculate cell color based on state
                if not node.walkable:
                    color = DARK_GRAY  # Obstacle
                elif (row, col) in [(drone.row, drone.col) for drone in self.drones]:
                    color = WHITE  # Drone position
                elif (row, col) == (self.target_row, self.target_col):
                    color = RED  # Target
                elif node.visited_by:
                    # Visited by multiple drones - mix colors
                    if len(node.visited_by) == 1:
                        drone_id = list(node.visited_by)[0]
                        base_color = DRONE_COLORS[drone_id % len(DRONE_COLORS)]
                        # Lighter version of the drone color
                        color = tuple(min(c + 100, 255) for c in base_color)
                    else:
                        color = YELLOW  # Multiple drones visited
                else:
                    # Check if the cell has been seen by any drone
                    cell_pos = (row, col)
                    seen = False
                    for drone in self.drones:
                        if cell_pos in drone.seen_cells:
                            seen = True
                            break
                    
                    if seen:
                        color = LIGHT_BLUE  # Seen but not visited
                    else:
                        color = BLACK  # Unexplored
                
                pygame.draw.rect(self.screen, color, (x, y, GRID_SIZE, GRID_SIZE))
                pygame.draw.rect(self.screen, GRAY, (x, y, GRID_SIZE, GRID_SIZE), 1)
        
        # Draw target
        target_x = int((self.target_col + 0.5) * GRID_SIZE)
        target_y = int((self.target_row + 0.5) * GRID_SIZE)
        pygame.draw.circle(self.screen, RED, (target_x, target_y), GRID_SIZE // 3)
        
        # Draw drones
        for drone in self.drones:
            drone.draw(self.screen, GRID_SIZE)
        
        # Display simulation stats
        time_text = self.font.render(f"Time: {self.simulation_time:.1f}s", True, WHITE)
        self.screen.blit(time_text, (10, 10))
        
        status_text = "Target found!" if self.target_found else "Searching..."
        status_render = self.font.render(status_text, True, GREEN if self.target_found else WHITE)
        self.screen.blit(status_render, (10, 40))
        
        coverage = len(set().union(*[drone.seen_cells for drone in self.drones])) / (ROWS * COLS) * 100
        coverage_text = self.font.render(f"Map Coverage: {coverage:.1f}%", True, WHITE)
        self.screen.blit(coverage_text, (10, 70))
        
        # Instructions
        instructions = self.font.render("Press R to reset simulation", True, WHITE)
        self.screen.blit(instructions, (WIDTH - 250, 10))
        
        # Update the display
        pygame.display.flip()
    
    def run(self):
        while self.running:
            dt = self.clock.tick(FPS) / 1000.0  # Time elapsed in seconds
            
            self.handle_events()
            self.update(dt)
            self.draw()
        
        pygame.quit()

if __name__ == "__main__":
    game = Game()
    game.run()

In [ ]:
# 3D formasyon
import pygame
import sys
import math
import time
from enum import Enum
import random
import os
import numpy as np
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *

# Formation types
class Formation(Enum):
    RANDOM = 0
    LINE = 1
    V_SHAPE = 2
    ARROWHEAD = 3
    CUBE = 4        # New 3D formation
    SPHERE = 5      # New 3D formation
    HELIX = 6       # New 3D formation

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
GRAY = (200, 200, 200)

# OpenGL colors (RGB format 0-1)
GL_WHITE = (1.0, 1.0, 1.0)
GL_BLACK = (0.0, 0.0, 0.0)
GL_RED = (1.0, 0.0, 0.0)
GL_GREEN = (0.0, 1.0, 0.0)
GL_BLUE = (0.0, 0.0, 1.0)
GL_GRAY = (0.8, 0.8, 0.8)

# Simulation parameters
NUM_DRONES = 10  # At least 3 drones
DRONE_SPACING = 50  # Meters (distance between drones)
MAX_ALTITUDE = 150  # Meters (flight altitude)
FORMATION_TIME = 10  # Seconds (formation holding time)

# Transition speeds
ALTITUDE_CHANGE_SPEED = 100  # Meters/second
FORMATION_TRANSITION_SPEED = 100  # Meters/second

# Camera and view settings
CAMERA_DISTANCE = 500
CAMERA_ROTATION_SPEED = 0.5

# Drone class for 3D simulation
class Drone:
    def __init__(self, id, x, y, z=0):
        self.id = id
        self.x = x
        self.y = y
        self.z = z  # Height/altitude
        self.target_x = x
        self.target_y = y
        self.target_z = z
        self.size = 2.0
        # Generate random color for the drone
        self.color = (random.random() * 0.5 + 0.5,
                     random.random() * 0.5 + 0.5,
                     random.random() * 0.5 + 0.5)
        
    def update(self, dt):
        # Move towards target position
        dx = self.target_x - self.x
        dy = self.target_y - self.y
        dz = self.target_z - self.z
        
        if abs(dx) > 0.1:
            self.x += min(FORMATION_TRANSITION_SPEED * dt, abs(dx)) * (1 if dx > 0 else -1)
        if abs(dy) > 0.1:
            self.y += min(FORMATION_TRANSITION_SPEED * dt, abs(dy)) * (1 if dy > 0 else -1)
        if abs(dz) > 0.1:
            self.z += min(ALTITUDE_CHANGE_SPEED * dt, abs(dz)) * (1 if dz > 0 else -1)
    
    def draw(self):
        glPushMatrix()
        glTranslatef(self.x, self.y, self.z)
        
        # Draw drone body
        glColor3f(*self.color)
        self.draw_cube(self.size)
        
        # Draw shadow (projected on ground)
        glPushMatrix()
        glTranslatef(0, 0, -self.z)
        glColor4f(0.0, 0.0, 0.0, max(0.1, 0.5 - self.z / 200))
        glScalef(1.0, 1.0, 0.01)
        self.draw_cube(self.size)
        glPopMatrix()
        
        glPopMatrix()
    
    def draw_cube(self, size):
        glBegin(GL_QUADS)
        # Front face
        glVertex3f(-size, -size, size)
        glVertex3f(size, -size, size)
        glVertex3f(size, size, size)
        glVertex3f(-size, size, size)
        
        # Back face
        glVertex3f(-size, -size, -size)
        glVertex3f(-size, size, -size)
        glVertex3f(size, size, -size)
        glVertex3f(size, -size, -size)
        
        # Top face
        glVertex3f(-size, size, -size)
        glVertex3f(-size, size, size)
        glVertex3f(size, size, size)
        glVertex3f(size, size, -size)
        
        # Bottom face
        glVertex3f(-size, -size, -size)
        glVertex3f(size, -size, -size)
        glVertex3f(size, -size, size)
        glVertex3f(-size, -size, size)
        
        # Right face
        glVertex3f(size, -size, -size)
        glVertex3f(size, size, -size)
        glVertex3f(size, size, size)
        glVertex3f(size, -size, size)
        
        # Left face
        glVertex3f(-size, -size, -size)
        glVertex3f(-size, -size, size)
        glVertex3f(-size, size, size)
        glVertex3f(-size, size, -size)
        glEnd()

# Drone Formation Manager for 3D
class FormationManager:
    def __init__(self, num_drones):
        self.drones = []
        self.current_formation = Formation.RANDOM
        self.target_formation = Formation.RANDOM
        self.formation_start_time = 0
        self.formation_complete = False
        self.mission_complete = False
        self.mission_step = 0
        self.missions = []
        
        # Create drones in initial random positions on the ground
        ground_y = 0
        formation_center = np.array([0, 0, 0])
        
        for i in range(num_drones):
            x = random.uniform(-100, 100)
            z = random.uniform(-100, 100)
            self.drones.append(Drone(i+1, x, ground_y, z))
    
    def set_formation(self, formation_type):
        self.target_formation = formation_type
        self.formation_complete = False
        self.formation_start_time = 0
        
        formation_center = np.array([0, 100, 0])  # Center point for formation
        
        if formation_type == Formation.RANDOM:
            # Random formation at altitude
            for i, drone in enumerate(self.drones):
                drone.target_x = random.uniform(-100, 100)
                drone.target_y = 100  # Fixed altitude for now
                drone.target_z = random.uniform(-100, 100)
                
        elif formation_type == Formation.LINE:
            # Line formation along X-axis
            total_width = (len(self.drones) - 1) * DRONE_SPACING
            start_x = -total_width / 2
            
            for i, drone in enumerate(self.drones):
                drone.target_x = start_x + i * DRONE_SPACING
                drone.target_y = formation_center[1]
                drone.target_z = formation_center[2]
                
        elif formation_type == Formation.V_SHAPE:
            # V formation in XZ plane
            angle = math.pi / 6  # 30 degrees
            
            for i, drone in enumerate(self.drones):
                if i == 0:  # Lead drone
                    drone.target_x = formation_center[0]
                    drone.target_y = formation_center[1]
                    drone.target_z = formation_center[2] - DRONE_SPACING
                else:
                    # Alternate between left and right sides
                    side = 1 if i % 2 == 1 else -1
                    position = (i + 1) // 2
                    
                    drone.target_x = formation_center[0] + side * position * DRONE_SPACING * math.sin(angle)
                    drone.target_y = formation_center[1]
                    drone.target_z = formation_center[2] + position * DRONE_SPACING * math.cos(angle)
                    
        elif formation_type == Formation.ARROWHEAD:
            # Arrowhead (Lambda Λ) formation in XZ plane
            angle = math.pi / 4  # 45 degrees
            
            for i, drone in enumerate(self.drones):
                if i == 0:  # Lead drone at the front
                    drone.target_x = formation_center[0]
                    drone.target_y = formation_center[1]
                    drone.target_z = formation_center[2] - DRONE_SPACING
                else:
                    # Alternate between left and right sides
                    side = 1 if i % 2 == 1 else -1
                    position = (i + 1) // 2
                    
                    drone.target_x = formation_center[0] + side * position * DRONE_SPACING * math.sin(angle)
                    drone.target_y = formation_center[1]
                    drone.target_z = formation_center[2] + position * DRONE_SPACING * math.cos(angle)
                    
        elif formation_type == Formation.CUBE:
            # 3D cube formation
            # Calculate cube dimensions based on number of drones
            cube_side = math.ceil(num_drones ** (1/3))
            spacing = DRONE_SPACING / 2  # Tighter spacing for cube
            
            # Calculate offsets to center the cube
            offset_x = -((cube_side - 1) * spacing) / 2
            offset_y = -((cube_side - 1) * spacing) / 2
            offset_z = -((cube_side - 1) * spacing) / 2
            
            drones_placed = 0
            for x in range(cube_side):
                for y in range(cube_side):
                    for z in range(cube_side):
                        if drones_placed < len(self.drones):
                            drone = self.drones[drones_placed]
                            drone.target_x = formation_center[0] + offset_x + x * spacing
                            drone.target_y = formation_center[1] + offset_y + y * spacing
                            drone.target_z = formation_center[2] + offset_z + z * spacing
                            drones_placed += 1
        
        elif formation_type == Formation.SPHERE:
            # 3D sphere formation
            radius = DRONE_SPACING * 2
            
            # Use Fibonacci sphere distribution for evenly spaced points on a sphere
            golden_ratio = (1 + 5**0.5) / 2
            
            for i, drone in enumerate(self.drones):
                # Calculate spherical coordinates
                i_normalized = i / (len(self.drones) - 1) if len(self.drones) > 1 else 0
                theta = 2 * math.pi * i / golden_ratio
                phi = math.acos(1 - 2 * i_normalized)
                
                # Convert to Cartesian coordinates
                drone.target_x = formation_center[0] + radius * math.sin(phi) * math.cos(theta)
                drone.target_y = formation_center[1] + radius * math.sin(phi) * math.sin(theta)
                drone.target_z = formation_center[2] + radius * math.cos(phi)
                
        elif formation_type == Formation.HELIX:
            # Helix (spiral) formation
            radius = DRONE_SPACING
            height_per_turn = DRONE_SPACING * 2
            turns = 2
            
            for i, drone in enumerate(self.drones):
                angle = 2 * math.pi * i / len(self.drones) * turns
                height_offset = height_per_turn * (i / len(self.drones)) * turns
                
                drone.target_x = formation_center[0] + radius * math.cos(angle)
                drone.target_y = formation_center[1] + height_offset
                drone.target_z = formation_center[2] + radius * math.sin(angle)
    
    def set_altitude(self, altitude):
        for drone in self.drones:
            drone.target_y = altitude
    
    def update(self, dt):
        # Update all drones
        for drone in self.drones:
            drone.update(dt)
        
        # Check if formation transition is complete
        all_in_position = True
        for drone in self.drones:
            if (abs(drone.x - drone.target_x) > 0.5 or 
                abs(drone.y - drone.target_y) > 0.5 or 
                abs(drone.z - drone.target_z) > 0.5):
                all_in_position = False
                break
        
        # If all drones are in position but we haven't started timing yet
        if all_in_position and not self.formation_complete and self.formation_start_time == 0:
            self.formation_start_time = time.time()
            print(f"Formation achieved. Holding for {FORMATION_TIME} seconds...")
            
        # Check if we've maintained formation for the required time
        if self.formation_start_time > 0 and time.time() - self.formation_start_time >= FORMATION_TIME:
            if not self.formation_complete:
                self.formation_complete = True
                self.current_formation = self.target_formation
                print("Formation holding time complete!")
                self.advance_mission()
    
    def draw(self):
        # Draw ground grid
        self.draw_ground_grid(20, 300)
        
        # Draw drones
        for drone in self.drones:
            drone.draw()
    
    def draw_ground_grid(self, grid_size, extent):
        glBegin(GL_LINES)
        glColor3f(0.5, 0.5, 0.5)
        
        for i in range(-extent, extent + 1, grid_size):
            # Draw lines along X axis
            glVertex3f(i, 0, -extent)
            glVertex3f(i, 0, extent)
            
            # Draw lines along Z axis
            glVertex3f(-extent, 0, i)
            glVertex3f(extent, 0, i)
        
        glEnd()
    
    def setup_mission(self, mission_steps):
        self.missions = mission_steps
        self.mission_step = 0
        self.mission_complete = False
        self.start_current_mission_step()
    
    def start_current_mission_step(self):
        if self.mission_step < len(self.missions):
            current_mission = self.missions[self.mission_step]
            
            if 'altitude' in current_mission:
                self.set_altitude(current_mission['altitude'])
            
            if 'formation' in current_mission:
                self.set_formation(current_mission['formation'])
            
            print(f"Starting mission step {self.mission_step + 1}: {current_mission['description']}")
    
    def advance_mission(self):
        self.mission_step += 1
        
        if self.mission_step < len(self.missions):
            self.start_current_mission_step()
        else:
            self.mission_complete = True
            print("Mission completed successfully!")

# Function to render text in 3D space
def render_text(position, text_string, color=GL_WHITE):
    # This is a placeholder - for actual text in 3D space, you would need
    # to use a texture-based approach or a library like pygame-menu
    # In this example, we'll draw a colored marker instead
    glPushMatrix()
    glTranslatef(*position)
    glColor3f(*color)
    glBegin(GL_POINTS)
    glVertex3f(0, 0, 0)
    glEnd()
    glPopMatrix()

def setup_opengl(self):
    glViewport(0, 0, WIDTH, HEIGHT)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(45, (WIDTH / HEIGHT), 0.1, 200.0)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
    # Enable 3D features
    glEnable(GL_DEPTH_TEST)
    glEnable(GL_BLEND)
    glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
    
    # Set background color to a light color
    glClearColor(0.9, 0.9, 0.9, 1.0)
    
    # Enable lighting for better 3D perception
    glEnable(GL_LIGHTING)
    glEnable(GL_LIGHT0)
    glEnable(GL_COLOR_MATERIAL)
    glColorMaterial(GL_FRONT_AND_BACK, GL_AMBIENT_AND_DIFFUSE)
    
    # Set up light position
    light_position = [COLS/2, ROWS*2, LAYERS/2, 1.0]
    glLightfv(GL_LIGHT0, GL_POSITION, light_position)
    
    # Increase light intensity
    light_ambient = [0.4, 0.4, 0.4, 1.0]
    light_diffuse = [0.8, 0.8, 0.8, 1.0]
    light_specular = [1.0, 1.0, 1.0, 1.0]
    glLightfv(GL_LIGHT0, GL_AMBIENT, light_ambient)
    glLightfv(GL_LIGHT0, GL_DIFFUSE, light_diffuse)
    glLightfv(GL_LIGHT0, GL_SPECULAR, light_specular)

# Main function
def main():
    pygame.init()
    
    # Set up the display with OpenGL
    screen_width, screen_height = 1024, 768
    pygame.display.set_mode((screen_width, screen_height), DOUBLEBUF | OPENGL)
    pygame.display.set_caption("3D Drone Swarm Formation Simulator")
    
    setup_opengl(screen_width, screen_height)
    
    # Setup 2D overlay for text
    overlay = pygame.Surface((screen_width, screen_height), pygame.SRCALPHA)
    font = pygame.font.SysFont('Arial', 18)
    
    # Create formation manager
    manager = FormationManager(NUM_DRONES)
    
    # Set up camera
    camera_angle_x = 0
    camera_angle_y = 30  # Initial camera tilt
    camera_distance = CAMERA_DISTANCE
    
    # Set up mission
    mission_steps = [
        {
            'description': "Drones ascending to operational altitude",
            'altitude': 100,
            'formation': Formation.RANDOM
        },
        {
            'description': 'Forming arrowhead formation',
            'formation': Formation.ARROWHEAD
        },
        {
            'description': 'Transitioning to line formation',
            'formation': Formation.LINE
        },
        {
            'description': 'Forming 3D cube formation',
            'formation': Formation.CUBE
        },
        {
            'description': 'Transitioning to sphere formation',
            'formation': Formation.SPHERE
        },
        {
            'description': 'Forming helix formation',
            'formation': Formation.HELIX
        },
        {
            'description': 'Returning to ground while maintaining formation',
            'altitude': 0
        }
    ]
    
    manager.setup_mission(mission_steps)
    
    # Main game loop
    clock = pygame.time.Clock()
    running = True
    last_time = time.time()
    autorotate = True
    
    while running:
        current_time = time.time()
        dt = current_time - last_time
        last_time = current_time
        
        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_r:
                    # Reset simulation
                    manager = FormationManager(NUM_DRONES)
                    manager.setup_mission(mission_steps)
                elif event.key == pygame.K_SPACE:
                    # Toggle autorotate
                    autorotate = not autorotate
                    
        # Camera control with mouse
        if pygame.mouse.get_pressed()[0]:  # Left mouse button
            rel_x, rel_y = pygame.mouse.get_rel()
            camera_angle_x += rel_x * 0.5
            camera_angle_y -= rel_y * 0.5
            camera_angle_y = max(-90, min(90, camera_angle_y))  # Clamp vertical angle
            autorotate = False
        else:
            pygame.mouse.get_rel()  # Clear relative movement
        
        # Autorotate camera
        if autorotate:
            camera_angle_x += CAMERA_ROTATION_SPEED
        
        # Update formation
        manager.update(dt)
        
        # Clear the screen and depth buffer
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glClearColor(0.1, 0.1, 0.2, 1.0)  # Dark blue background
        
        # Setup camera position
        glLoadIdentity()
        
        # Calculate camera position based on spherical coordinates
        cam_x = camera_distance * math.cos(math.radians(camera_angle_y)) * math.sin(math.radians(camera_angle_x))
        cam_y = camera_distance * math.sin(math.radians(camera_angle_y))
        cam_z = camera_distance * math.cos(math.radians(camera_angle_y)) * math.cos(math.radians(camera_angle_x))
        
        # Set camera position and target
        gluLookAt(cam_x, cam_y, cam_z,  # Camera position
                 0, 50, 0,              # Target (look at center, slightly above ground)
                 0, 1, 0)               # Up vector
        
        # Draw the 3D scene
        manager.draw()
        
        # Draw 2D overlay
        overlay.fill((0, 0, 0, 0))  # Clear with transparent background
        
        # Display formation info
        formation_names = {
            Formation.RANDOM: "Random",
            Formation.LINE: "Line",
            Formation.V_SHAPE: "V-Shape",
            Formation.ARROWHEAD: "Arrowhead",
            Formation.CUBE: "Cube",
            Formation.SPHERE: "Sphere",
            Formation.HELIX: "Helix"
        }
        
        formation_text = f"Formation: {formation_names[manager.target_formation]}"
        formation_surface = font.render(formation_text, True, WHITE)
        overlay.blit(formation_surface, (20, 20))
        
        if manager.formation_start_time > 0 and not manager.formation_complete:
            elapsed = time.time() - manager.formation_start_time
            remaining = max(0, FORMATION_TIME - elapsed)
            time_text = f"Holding: {remaining:.1f}s remaining"
            time_surface = font.render(time_text, True, BLUE)
            overlay.blit(time_surface, (20, 50))
        
        if manager.mission_step < len(manager.missions):
            mission_text = f"Step {manager.mission_step + 1}: {manager.missions[manager.mission_step]['description']}"
            mission_surface = font.render(mission_text, True, WHITE)
            overlay.blit(mission_surface, (20, 80))
        
        if manager.mission_complete:
            complete_text = "Mission Complete!"
            complete_surface = font.render(complete_text, True, GREEN)
            overlay.blit(complete_surface, (screen_width//2 - 100, 20))
        
        # Instructions
        controls_text = "Controls: Left-click and drag to rotate view, SPACE to toggle auto-rotation, R to restart, ESC to quit"
        controls_surface = font.render(controls_text, True, WHITE)
        overlay.blit(controls_surface, (20, screen_height - 30))
        
        # Draw overlay on screen
        pygame.display.get_surface().blit(overlay, (0, 0))
        
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

In [ ]:
# 3D BFS
import pygame
import random
import math
import heapq
import numpy as np
from collections import deque
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *

# Constants
WIDTH, HEIGHT = 1024, 768
GRID_SIZE = 5  # Size of each grid cell
GRID_DIMS = (20, 10, 20)  # 3D grid dimensions (x, y, z)
COLS, ROWS, LAYERS = GRID_DIMS
FPS = 30

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (200, 200, 200)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
DRONE_COLORS = [(0, 0, 255), (0, 150, 255), (100, 100, 255)]
LIGHT_BLUE = (173, 216, 230)
DARK_GRAY = (50, 50, 50)

# OpenGL colors (RGB format 0-1)
GL_WHITE = (1.0, 1.0, 1.0)
GL_BLACK = (0.0, 0.0, 0.0)
GL_RED = (1.0, 0.0, 0.0)
GL_GREEN = (0.0, 1.0, 0.0)
GL_BLUE = (0.0, 0.0, 1.0)
GL_YELLOW = (1.0, 1.0, 0.0)
GL_GRAY = (0.8, 0.8, 0.8)
GL_DARK_GRAY = (0.2, 0.2, 0.2)
GL_LIGHT_BLUE = (0.678, 0.847, 0.902)

# Drone properties
DRONE_RADIUS = 0.3
EXPLORATION_RADIUS = 3  # Grid cells
DRONE_SPEED = 2  # Grid cells per second
DRONE_SCAN_FREQ = 1  # Seconds between scans

# Camera and view settings
CAMERA_DISTANCE = 40
CAMERA_ROTATION_SPEED = 0.5

class Node:
    def __init__(self, x, y, z, walkable=True):
        self.x = x
        self.y = y
        self.z = z
        self.walkable = walkable
        self.visited = False
        self.path = False
        self.visited_by = set()  # Set of drone IDs that have visited this node
        self.g_score = float('inf')
        self.f_score = float('inf')
        self.parent = None
        self.scanned = False  # Whether this node has been scanned by any drone
    
    def get_pos(self):
        return (self.x, self.y, self.z)
    
    def reset_path(self):
        self.path = False
        self.g_score = float('inf')
        self.f_score = float('inf')
        self.parent = None

class Grid:
    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth
        self.grid = [[[Node(x, y, z) for z in range(depth)] for y in range(height)] for x in range(width)]
        self.obstacles = set()
    
    def get_node(self, x, y, z):
        if 0 <= x < self.width and 0 <= y < self.height and 0 <= z < self.depth:
            return self.grid[x][y][z]
        return None
    
    def get_neighbors(self, node):
        neighbors = []
        # 6-connectivity in 3D space (face neighbors)
        for dx, dy, dz in [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]:
            nx, ny, nz = node.x + dx, node.y + dy, node.z + dz
            if 0 <= nx < self.width and 0 <= ny < self.height and 0 <= nz < self.depth:
                if self.grid[nx][ny][nz].walkable:
                    neighbors.append(self.grid[nx][ny][nz])
        return neighbors
    
    def reset_paths(self):
        for x in range(self.width):
            for y in range(self.height):
                for z in range(self.depth):
                    self.grid[x][y][z].reset_path()
    
    def add_random_obstacles(self, obstacle_ratio=0.15):
        num_obstacles = int(self.width * self.height * self.depth * obstacle_ratio)
        
        # Clear existing obstacles
        for x in range(self.width):
            for y in range(self.height):
                for z in range(self.depth):
                    self.grid[x][y][z].walkable = True
        
        self.obstacles.clear()
        
        # Add new obstacles
        for _ in range(num_obstacles):
            x = random.randint(1, self.width - 2)
            y = random.randint(1, self.height - 2)
            z = random.randint(1, self.depth - 2)
            
            # Create small obstacle clusters for more realistic environment
            cluster_size = random.randint(1, 3)
            for cx in range(x, min(x + cluster_size, self.width - 1)):
                for cy in range(y, min(y + cluster_size, self.height - 1)):
                    for cz in range(z, min(z + cluster_size, self.depth - 1)):
                        if random.random() < 0.7:  # 70% chance to add obstacle in cluster
                            node = self.get_node(cx, cy, cz)
                            if node:
                                node.walkable = False
                                self.obstacles.add((cx, cy, cz))

class Drone:
    def __init__(self, grid, id, x, y, z):
        self.grid = grid
        self.id = id
        self.x = x
        self.y = y
        self.z = z
        self.target_x = x
        self.target_y = y
        self.target_z = z
        self.color = self.get_color_from_id(id)
        self.path = []
        self.scan_timer = 0
        self.seen_cells = set()  # Cells this drone has seen
        self.target_found = False
        self.target_location = None
        self.move_fraction = 0.0
        self.direction = (0, 0, 0)
        self.explored_nodes = set()
    
    def get_color_from_id(self, id):
        base_colors = [
            (0.0, 0.0, 1.0),  # Blue
            (0.0, 0.6, 1.0),  # Light Blue
            (0.4, 0.4, 1.0),  # Purple-Blue
            (1.0, 0.0, 0.0),  # Red
            (0.0, 0.8, 0.0)   # Green
        ]
        return base_colors[id % len(base_colors)]
    
    def update(self, dt, global_visited, target_pos, shared_target_found):
        self.scan_timer += dt

        # Current position as a node
        current_pos = (self.x, self.y, self.z)
        target_pos_node = (self.target_x, self.target_y, self.target_z)
        
        # If we don't have a path or we're at the target position, find a new path
        if (not self.path or (current_pos == target_pos_node)) and not self.target_found:
            if shared_target_found:
                # If another drone found the target, set that as our goal
                self.path = self.find_path(target_pos[0], target_pos[1], target_pos[2])
                self.target_found = True
                self.target_location = target_pos
            else:
                # Otherwise explore
                self.explore(global_visited)

        # Move along path if we have one
        if self.path:
            self.move_along_path(dt)
        
        # Scan surroundings periodically
        if self.scan_timer >= DRONE_SCAN_FREQ:
            self.scan()
            self.scan_timer = 0
        
        # Check if we found target
        if not self.target_found and target_pos:
            distance = math.sqrt((self.x - target_pos[0])**2 + 
                                (self.y - target_pos[1])**2 + 
                                (self.z - target_pos[2])**2)
            if distance <= EXPLORATION_RADIUS:
                self.target_found = True
                self.target_location = target_pos
                self.path = self.find_path(target_pos[0], target_pos[1], target_pos[2])
        
        # Update cells we've visited
        current_node = self.grid.get_node(self.x, self.y, self.z)
        if current_node:
            # Mark this node as visited by this drone
            current_node.visited_by.add(self.id)
            self.explored_nodes.add((self.x, self.y, self.z))

    def scan(self):
        # Scan cells within exploration radius in 3D
        for x in range(self.x - EXPLORATION_RADIUS, self.x + EXPLORATION_RADIUS + 1):
            for y in range(self.y - EXPLORATION_RADIUS, self.y + EXPLORATION_RADIUS + 1):
                for z in range(self.z - EXPLORATION_RADIUS, self.z + EXPLORATION_RADIUS + 1):
                    # Check if cell is within grid bounds
                    if (0 <= x < self.grid.width and 
                        0 <= y < self.grid.height and 
                        0 <= z < self.grid.depth):
                        
                        # Check if within spherical exploration radius
                        distance = math.sqrt((x - self.x)**2 + (y - self.y)**2 + (z - self.z)**2)
                        if distance <= EXPLORATION_RADIUS:
                            self.seen_cells.add((x, y, z))
                            node = self.grid.get_node(x, y, z)
                            if node:
                                node.scanned = True
    
    def explore(self, global_visited):
        # Use BFS to find the nearest unexplored cell
        frontier = deque([(self.x, self.y, self.z)])
        came_from = {(self.x, self.y, self.z): None}
        
        target_cell = None
        
        while frontier and not target_cell:
            current = frontier.popleft()
            current_x, current_y, current_z = current
            
            # Check if this cell is unexplored globally and can be reached
            if current not in global_visited and self.grid.get_node(current_x, current_y, current_z).walkable:
                target_cell = current
                break
            
            # Add neighbors to frontier
            for dx, dy, dz in [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]:
                neighbor_x, neighbor_y, neighbor_z = current_x + dx, current_y + dy, current_z + dz
                neighbor = (neighbor_x, neighbor_y, neighbor_z)
                
                # Check if neighbor is in grid and not visited in BFS yet
                if (0 <= neighbor_x < self.grid.width and 
                    0 <= neighbor_y < self.grid.height and 
                    0 <= neighbor_z < self.grid.depth and 
                    neighbor not in came_from and
                    self.grid.get_node(neighbor_x, neighbor_y, neighbor_z).walkable):
                    frontier.append(neighbor)
                    came_from[neighbor] = current
        
        # If we found an unexplored cell, reconstruct path to it
        if target_cell:
            # Reconstruct path
            path = []
            current = target_cell
            while current != (self.x, self.y, self.z):
                path.append(current)
                current = came_from[current]
            path.reverse()
            
            # Set as target
            self.target_x, self.target_y, self.target_z = target_cell
            self.path = path
        else:
            # If no unexplored cells, just wander
            self.wander()
    
    def wander(self):
        # Pick a random walkable cell as a target
        walkable_cells = []
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                for z in range(self.grid.depth):
                    node = self.grid.get_node(x, y, z)
                    if node and node.walkable:
                        walkable_cells.append((x, y, z))
        
        if walkable_cells:
            target = random.choice(walkable_cells)
            self.target_x, self.target_y, self.target_z = target
            self.path = self.find_path(target[0], target[1], target[2])
    
    def find_path(self, target_x, target_y, target_z):
        # A* pathfinding in 3D
        start_node = self.grid.get_node(self.x, self.y, self.z)
        end_node = self.grid.get_node(target_x, target_y, target_z)
        
        if not start_node or not end_node or not end_node.walkable:
            return []
        
        # Reset previous path calculations
        self.grid.reset_paths()
        
        open_set = []
        closed_set = set()
        
        start_node.g_score = 0
        start_node.f_score = self.heuristic(start_node, end_node)
        heapq.heappush(open_set, (start_node.f_score, id(start_node), start_node))
        
        while open_set:
            _, _, current_node = heapq.heappop(open_set)
            
            if current_node == end_node:
                # Reconstruct path
                path = []
                while current_node != start_node:
                    path.append((current_node.x, current_node.y, current_node.z))
                    current_node = current_node.parent
                path.reverse()
                return path
            
            closed_set.add(current_node)
            
            for neighbor in self.grid.get_neighbors(current_node):
                if neighbor in closed_set:
                    continue
                
                # Calculate g_score (path distance from start to this neighbor through current)
                tentative_g_score = current_node.g_score + self.distance(current_node, neighbor)
                
                # If this path is better than previous ones
                if tentative_g_score < neighbor.g_score:
                    neighbor.parent = current_node
                    neighbor.g_score = tentative_g_score
                    neighbor.f_score = tentative_g_score + self.heuristic(neighbor, end_node)
                    neighbor.path = True
                    
                    # Add to open set if not already there
                    if all(n != neighbor for _, _, n in open_set):
                        heapq.heappush(open_set, (neighbor.f_score, id(neighbor), neighbor))
        
        # No path found
        return []
    
    def heuristic(self, node, target):
        # Manhattan distance in 3D
        return (abs(node.x - target.x) + 
                abs(node.y - target.y) + 
                abs(node.z - target.z))
    
    def distance(self, node1, node2):
        # Euclidean distance in 3D for A*
        return math.sqrt((node1.x - node2.x)**2 + 
                        (node1.y - node2.y)**2 + 
                        (node1.z - node2.z)**2)
    
    def move_along_path(self, dt):
        if not self.path:
            return
        
        if self.move_fraction == 0:
            # Starting a new movement segment
            next_pos = self.path[0]
            self.direction = (next_pos[0] - self.x, next_pos[1] - self.y, next_pos[2] - self.z)
        
        # Update movement progress
        move_speed = DRONE_SPEED * dt
        self.move_fraction += move_speed
        
        if self.move_fraction >= 1.0:
            # Completed this segment
            self.x, self.y, self.z = self.path.pop(0)
            self.move_fraction = 0.0
            self.direction = (0, 0, 0)
        
    def draw(self):
        # Calculate interpolated position based on move_fraction
        display_x = self.x
        display_y = self.y
        display_z = self.z
        
        if self.move_fraction > 0 and self.path:
            next_x, next_y, next_z = self.path[0]
            display_x = self.x + (next_x - self.x) * self.move_fraction
            display_y = self.y + (next_y - self.y) * self.move_fraction
            display_z = self.z + (next_z - self.z) * self.move_fraction
        
        # Draw the drone as a sphere
        glPushMatrix()
        glTranslatef(display_x, display_y, display_z)
        
        # Set drone color
        glColor3f(*self.color)
        
        # Draw drone body (sphere)
        quadric = gluNewQuadric()
        gluSphere(quadric, DRONE_RADIUS, 12, 12)
        gluDeleteQuadric(quadric)
        
        # Draw exploration radius (wireframe sphere)
        if self.id == 0:  # Only draw for first drone to reduce visual clutter
            glColor4f(*self.color, 0.3)  # Transparent
            glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
            
            quadric = gluNewQuadric()
            gluSphere(quadric, EXPLORATION_RADIUS, 12, 12)
            gluDeleteQuadric(quadric)
            
            glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)
        
        glPopMatrix()
        
        # Draw path
        if self.path and len(self.path) > 0:
            glBegin(GL_LINE_STRIP)
            glColor3f(*self.color)
            
            # Start from current position
            glVertex3f(display_x, display_y, display_z)
            
            # Add all path points
            for x, y, z in self.path:
                glVertex3f(x, y, z)
            
            glEnd()
    def draw_grid(self):
        # Draw grid structure - only draw specific cells based on their state
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                for z in range(self.grid.depth):
                    node = self.grid.get_node(x, y, z)
                    
                    # Only draw cells that are:
                    # 1. Obstacles
                    # 2. Visited by drones
                    # 3. Target position
                    # 4. Cells with high exploration value
                    if not node.walkable:
                        # For obstacles, only draw those that are near explored areas or on the boundary
                        has_explored_neighbor = False
                        for dx, dy, dz in [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]:
                            nx, ny, nz = x + dx, y + dy, z + dz
                            if (0 <= nx < self.grid.width and 
                                0 <= ny < self.grid.height and 
                                0 <= nz < self.grid.depth):
                                neighbor = self.grid.get_node(nx, ny, nz)
                                if neighbor.scanned:
                                    has_explored_neighbor = True
                                    break
                        
                        # Only draw obstacle if it's near explored area
                        if has_explored_neighbor:
                            self.draw_cell(x, y, z, node)
                    elif (x, y, z) == (self.target_x, self.target_y, self.target_z):
                        # Always draw target
                        self.draw_cell(x, y, z, node)
                    elif node.visited_by:
                        # Always draw visited cells
                        self.draw_cell(x, y, z, node)
                    elif node.scanned:
                        # For scanned cells, only draw a fraction to reduce clutter
                        # The closer to a drone path, the more likely to draw
                        draw_probability = 0.3  # Base probability
                        
                        # Increase probability if near a drone
                        for drone in self.drones:
                            dist = math.sqrt((x - drone.x)**2 + (y - drone.y)**2 + (z - drone.z)**2)
                            if dist < EXPLORATION_RADIUS:
                                draw_probability += 0.4 * (1 - dist/EXPLORATION_RADIUS)
                        
                        if random.random() < draw_probability:
                            self.draw_cell(x, y, z, node)
                            
class Game:
    def __init__(self):
        pygame.init()
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT), DOUBLEBUF | OPENGL)
        pygame.display.set_caption("3D Drone Search Simulation")
        self.clock = pygame.time.Clock()
        self.running = True
        
        # Set up OpenGL
        self.setup_opengl()
        
        # Set up 2D overlay for text
        self.overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
        self.font = pygame.font.SysFont(None, 24)
        
        # Create 3D grid
        self.grid = Grid(COLS, ROWS, LAYERS)
        self.grid.add_random_obstacles(obstacle_ratio=0.1)  # Reduced obstacles
        
        # Place drones at random positions
        self.drones = []
        drone_positions = set()
        for i in range(3):
            while True:
                x = random.randint(0, COLS - 1)
                y = random.randint(0, ROWS - 1)
                z = random.randint(0, LAYERS - 1)
                if (x, y, z) not in drone_positions and (x, y, z) not in self.grid.obstacles:
                    drone_positions.add((x, y, z))
                    self.drones.append(Drone(self.grid, i, x, y, z))
                    break
        
        # Place target randomly
        while True:
            self.target_x = random.randint(0, COLS - 1)
            self.target_y = random.randint(0, ROWS - 1)
            self.target_z = random.randint(0, LAYERS - 1)
            target_pos = (self.target_x, self.target_y, self.target_z)
            if target_pos not in drone_positions and target_pos not in self.grid.obstacles:
                break
        
        self.target_found = False
        self.simulation_time = 0
        
        # Camera settings
        self.camera_angle_x = 45
        self.camera_angle_y = 30
        self.camera_distance = CAMERA_DISTANCE
        self.autorotate = True
    
    def setup_opengl(self):
        glViewport(0, 0, WIDTH, HEIGHT)
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        gluPerspective(45, (WIDTH / HEIGHT), 0.1, 200.0)
        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()
        
        # Enable 3D features
        glEnable(GL_DEPTH_TEST)
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
    
    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    self.running = False
                elif event.key == pygame.K_r:
                    # Reset simulation
                    self.__init__()
                elif event.key == pygame.K_SPACE:
                    # Toggle autorotate
                    self.autorotate = not self.autorotate
            
        # Camera control with mouse
        if pygame.mouse.get_pressed()[0]:  # Left mouse button
            rel_x, rel_y = pygame.mouse.get_rel()
            self.camera_angle_x += rel_x * 0.5
            self.camera_angle_y -= rel_y * 0.5
            self.camera_angle_y = max(-85, min(85, self.camera_angle_y))  # Clamp vertical angle
            self.autorotate = False
        else:
            pygame.mouse.get_rel()  # Clear relative movement
    
    def update(self, dt):
        self.simulation_time += dt
        
        # Autorotate camera
        if self.autorotate:
            self.camera_angle_x += CAMERA_ROTATION_SPEED * dt * 10
        
        # Gather global visited cells (combine all drones' seen cells)
        global_visited = set()
        for drone in self.drones:
            global_visited.update(drone.seen_cells)
        
        # Update all drones
        target_pos = (self.target_x, self.target_y, self.target_z) if self.target_found else None
        
        # Check if any drone found the target
        self.target_found = any(drone.target_found for drone in self.drones)
        
        for drone in self.drones:
            drone.update(dt, global_visited, (self.target_x, self.target_y, self.target_z), self.target_found)
            
            # If this drone found the target, share the information
            if drone.target_found and not self.target_found:
                self.target_found = True
    
    def draw_grid(self):
        # Draw grid structure and cells based on their state
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                for z in range(self.grid.depth):
                    node = self.grid.get_node(x, y, z)
                    
                    # Skip rendering empty space for better performance
                    if not node.walkable or node.scanned:
                        self.draw_cell(x, y, z, node)
    
    def draw_cell(self, x, y, z, node):
        cell_color = GL_WHITE
        alpha = 0.1  # Default transparency
        render_wireframe = True
        
        # Determine cell color and visibility based on state
        if not node.walkable:
            # Make obstacles less opaque
            cell_color = GL_DARK_GRAY
            alpha = 0.4  # Reduced from 0.8
            render_wireframe = False
        elif (x, y, z) == (self.target_x, self.target_y, self.target_z):
            # Target
            cell_color = GL_RED
            alpha = 0.9
            render_wireframe = False
        elif node.visited_by:
            # Visited by drones - use drone color
            if len(node.visited_by) == 1:
                drone_id = list(node.visited_by)[0]
                cell_color = self.drones[drone_id % len(self.drones)].color
                alpha = 0.6  # Increased from 0.4
            else:
                # Multiple drones visited
                cell_color = GL_YELLOW
                alpha = 0.7  # Increased from 0.5
        elif node.scanned:
            # Scanned but not visited
            cell_color = GL_LIGHT_BLUE
            alpha = 0.2
        else:
            # Skip rendering unexplored cells
            return
        
        # Temporarily disable lighting for wireframes
        if render_wireframe:
            glDisable(GL_LIGHTING)
        
        # Draw cell as a cube
        glPushMatrix()
        glTranslatef(x, y, z)
        
        # Set color with transparency
        glColor4f(cell_color[0], cell_color[1], cell_color[2], alpha)
        
        # Draw cube
        if render_wireframe:
            # Wireframe for scanned cells
            glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
            self.draw_cube(0.5)
            glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)
        else:
            # Solid for obstacles and target
            self.draw_cube(0.5)
        
        glPopMatrix()
        
        # Re-enable lighting
        if render_wireframe:
            glEnable(GL_LIGHTING)
    
    def draw_cube(self, size):
        glBegin(GL_QUADS)
        # Front face
        glVertex3f(-size, -size, size)
        glVertex3f(size, -size, size)
        glVertex3f(size, size, size)
        glVertex3f(-size, size, size)
        
        # Back face
        glVertex3f(-size, -size, -size)
        glVertex3f(-size, size, -size)
        glVertex3f(size, size, -size)
        glVertex3f(size, -size, -size)
        
        # Top face
        glVertex3f(-size, size, -size)
        glVertex3f(-size, size, size)
        glVertex3f(size, size, size)
        glVertex3f(size, size, -size)
        
        # Bottom face
        glVertex3f(-size, -size, -size)
        glVertex3f(size, -size, -size)
        glVertex3f(size, -size, size)
        glVertex3f(-size, -size, size)
        
        # Right face
        glVertex3f(size, -size, -size)
        glVertex3f(size, size, -size)
        glVertex3f(size, size, size)
        glVertex3f(size, -size, size)
        
        # Left face
        glVertex3f(-size, -size, -size)
        glVertex3f(-size, -size, size)
        glVertex3f(-size, size, size)
        glVertex3f(-size, size, -size)
        glEnd()
    
    def draw_target(self):
        # Temporarily disable lighting for rotating indicators
        glDisable(GL_LIGHTING)
        
        # Draw target position indicator before the target itself
        glPushMatrix()
        glTranslatef(self.target_x, self.target_y, self.target_z)
        
        # Draw a vertical line to make target more visible
        glLineWidth(3.0)
        glBegin(GL_LINES)
        glColor3f(*GL_YELLOW)
        glVertex3f(0, -5, 0)
        glVertex3f(0, self.target_y * 2, 0)
        glEnd()
        glLineWidth(1.0)
        
        # Add a rotating indicator for visibility
        glRotatef(self.simulation_time * 100, 0, 1, 0)
        
        glLineWidth(3.0)
        glBegin(GL_LINES)
        glColor3f(*GL_YELLOW)
        line_length = 1.5  # Longer lines
        glVertex3f(0, 0, 0)
        glVertex3f(line_length, 0, 0)
        glVertex3f(0, 0, 0)
        glVertex3f(0, line_length, 0)
        glVertex3f(0, 0, 0)
        glVertex3f(0, 0, line_length)
        glEnd()
        glLineWidth(1.0)
        
        glPopMatrix()
        
        # Re-enable lighting for the target sphere
        glEnable(GL_LIGHTING)
        
        # Draw target as a sphere
        glPushMatrix()
        glTranslatef(self.target_x, self.target_y, self.target_z)
        
        # Draw target as a sphere
        glColor3f(*GL_RED)
        quadric = gluNewQuadric()
        gluSphere(quadric, 0.8, 16, 16)  # Larger target sphere
        gluDeleteQuadric(quadric)
        
        glPopMatrix()
    
    def draw_boundaries(self):
        # Temporarily disable lighting for grid lines
        glDisable(GL_LIGHTING)
        
        # Draw grid boundaries with thicker lines
        glLineWidth(2.0)
        glBegin(GL_LINES)
        glColor3f(0.3, 0.3, 0.3)  # Darker color for contrast
        
        # Draw grid outline
        glVertex3f(0, 0, 0)
        glVertex3f(COLS, 0, 0)
        
        glVertex3f(0, 0, 0)
        glVertex3f(0, ROWS, 0)
        
        glVertex3f(0, 0, 0)
        glVertex3f(0, 0, LAYERS)
        
        glVertex3f(COLS, 0, 0)
        glVertex3f(COLS, ROWS, 0)
        
        glVertex3f(COLS, 0, 0)
        glVertex3f(COLS, 0, LAYERS)
        
        glVertex3f(0, ROWS, 0)
        glVertex3f(COLS, ROWS, 0)
        
        glVertex3f(0, ROWS, 0)
        glVertex3f(0, ROWS, LAYERS)
        
        glVertex3f(0, 0, LAYERS)
        glVertex3f(COLS, 0, LAYERS)
        
        glVertex3f(0, 0, LAYERS)
        glVertex3f(0, ROWS, LAYERS)
        
        glVertex3f(COLS, ROWS, 0)
        glVertex3f(COLS, ROWS, LAYERS)
        
        glVertex3f(COLS, 0, LAYERS)
        glVertex3f(COLS, ROWS, LAYERS)
        
        glVertex3f(0, ROWS, LAYERS)
        glVertex3f(COLS, ROWS, LAYERS)
        
        glEnd()
        glLineWidth(1.0)  # Reset line width
        
        # Re-enable lighting
        glEnable(GL_LIGHTING)

    
    def draw_info_overlay(self):
        # Reset the overlay surface
        self.overlay.fill((0, 0, 0, 0))
        
        # Display simulation info
        info_text = [
            f"Time: {self.simulation_time:.1f}s",
            f"Target Found: {self.target_found}",
            f"Drones: {len(self.drones)}",
            f"Explored: {sum(len(drone.explored_nodes) for drone in self.drones)} cells",
            f"Controls: ESC to quit, R to restart, SPACE to toggle rotation"
        ]
        
        for i, text in enumerate(info_text):
            text_surface = self.font.render(text, True, WHITE)
            self.overlay.blit(text_surface, (10, 10 + i * 25))
        
        # Display coverage information
        total_cells = self.grid.width * self.grid.height * self.grid.depth
        scanned_cells = sum(1 for x in range(self.grid.width) 
                            for y in range(self.grid.height) 
                            for z in range(self.grid.depth) 
                            if self.grid.get_node(x, y, z).scanned)
        
        coverage_pct = (scanned_cells / total_cells) * 100
        coverage_text = f"Coverage: {coverage_pct:.1f}%"
        text_surface = self.font.render(coverage_text, True, WHITE)
        self.overlay.blit(text_surface, (WIDTH - text_surface.get_width() - 10, 10))
        
        # If target is found, display time taken
        if self.target_found:
            found_text = f"Target found in {self.simulation_time:.2f} seconds!"
            text_surface = self.font.render(found_text, True, GREEN)
            self.overlay.blit(text_surface, 
                              (WIDTH // 2 - text_surface.get_width() // 2, HEIGHT - 40))
    
    def draw(self):
        # Clear the screen and depth buffer
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glLoadIdentity()
        
        # Position camera based on angles
        camera_x = self.camera_distance * math.sin(math.radians(self.camera_angle_x)) * math.cos(math.radians(self.camera_angle_y))
        camera_y = self.camera_distance * math.sin(math.radians(self.camera_angle_y))
        camera_z = self.camera_distance * math.cos(math.radians(self.camera_angle_x)) * math.cos(math.radians(self.camera_angle_y))
        
        # Set the camera position
        gluLookAt(
            camera_x + COLS/2, camera_y + ROWS/2, camera_z + LAYERS/2,  # Camera position
            COLS/2, ROWS/2, LAYERS/2,  # Look at center of grid
            0, 1, 0  # Up vector
        )
        
        # Draw grid boundaries
        self.draw_boundaries()
        
        # Draw grid cells
        self.draw_grid()
        
        # Draw target
        self.draw_target()
        
        # Draw all drones
        for drone in self.drones:
            drone.draw()
        
        # Reset to 2D mode for overlay
        glMatrixMode(GL_PROJECTION)
        glPushMatrix()
        glLoadIdentity()
        glOrtho(0, WIDTH, HEIGHT, 0, -1, 1)
        glMatrixMode(GL_MODELVIEW)
        glPushMatrix()
        glLoadIdentity()
        
        # Draw info overlay
        self.draw_info_overlay()
        self.screen.blit(self.overlay, (0, 0))
        
        # Restore 3D mode
        glMatrixMode(GL_PROJECTION)
        glPopMatrix()
        glMatrixMode(GL_MODELVIEW)
        glPopMatrix()
        
        pygame.display.flip()
    
    def run(self):
        last_time = pygame.time.get_ticks() / 1000.0
        
        while self.running:
            # Calculate time delta
            current_time = pygame.time.get_ticks() / 1000.0
            dt = current_time - last_time
            last_time = current_time
            
            # Handle events
            self.handle_events()
            
            # Update simulation
            self.update(dt)
            
            # Draw everything
            self.draw()
            
            # Cap the frame rate
            self.clock.tick(FPS)
        
        pygame.quit()
    def setup_opengl(self):
        glViewport(0, 0, WIDTH, HEIGHT)
        glMatrixMode(GL_PROJECTION)
        glLoadIdentity()
        gluPerspective(45, (WIDTH / HEIGHT), 0.1, 200.0)
        glMatrixMode(GL_MODELVIEW)
        glLoadIdentity()
        
        # Enable 3D features
        glEnable(GL_DEPTH_TEST)
        glEnable(GL_BLEND)
        glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
        
        # Set background color to a light color
        glClearColor(0.9, 0.9, 0.9, 1.0)
        
        # Enable lighting for better 3D perception
        glEnable(GL_LIGHTING)
        glEnable(GL_LIGHT0)
        glEnable(GL_COLOR_MATERIAL)
        glColorMaterial(GL_FRONT_AND_BACK, GL_AMBIENT_AND_DIFFUSE)
        
        # Set up light position
        light_position = [COLS/2, ROWS*2, LAYERS/2, 1.0]
        glLightfv(GL_LIGHT0, GL_POSITION, light_position)
        
        # Increase light intensity
        light_ambient = [0.4, 0.4, 0.4, 1.0]
        light_diffuse = [0.8, 0.8, 0.8, 1.0]
        light_specular = [1.0, 1.0, 1.0, 1.0]
        glLightfv(GL_LIGHT0, GL_AMBIENT, light_ambient)
        glLightfv(GL_LIGHT0, GL_DIFFUSE, light_diffuse)
        glLightfv(GL_LIGHT0, GL_SPECULAR, light_specular)
    
    def draw_cell(self, x, y, z, node):
        cell_color = GL_WHITE
        alpha = 0.1  # Default transparency
        render_wireframe = True
        
        # Determine cell color and visibility based on state
        if not node.walkable:
            # Make obstacles less opaque
            cell_color = GL_DARK_GRAY
            alpha = 0.4  # Reduced from 0.8
            render_wireframe = False
        elif (x, y, z) == (self.target_x, self.target_y, self.target_z):
            # Target
            cell_color = GL_RED
            alpha = 0.9
            render_wireframe = False
        elif node.visited_by:
            # Visited by drones - use drone color
            if len(node.visited_by) == 1:
                drone_id = list(node.visited_by)[0]
                cell_color = self.drones[drone_id % len(self.drones)].color
                alpha = 0.6  # Increased from 0.4
            else:
                # Multiple drones visited
                cell_color = GL_YELLOW
                alpha = 0.7  # Increased from 0.5
        elif node.scanned:
            # Scanned but not visited
            cell_color = GL_LIGHT_BLUE
            alpha = 0.2
        else:
            # Skip rendering unexplored cells
            return
        
        # Temporarily disable lighting for wireframes
        if render_wireframe:
            glDisable(GL_LIGHTING)
        
        # Draw cell as a cube
        glPushMatrix()
        glTranslatef(x, y, z)
        
        # Set color with transparency
        glColor4f(cell_color[0], cell_color[1], cell_color[2], alpha)
        
        # Draw cube
        if render_wireframe:
            # Wireframe for scanned cells
            glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
            self.draw_cube(0.5)
            glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)
        else:
            # Solid for obstacles and target
            self.draw_cube(0.5)
        
        glPopMatrix()
        
        # Re-enable lighting
        if render_wireframe:
            glEnable(GL_LIGHTING)
    
    def draw_boundaries(self):
        # Temporarily disable lighting for grid lines
        glDisable(GL_LIGHTING)
        
        # Draw grid boundaries with thicker lines
        glLineWidth(2.0)
        glBegin(GL_LINES)
        glColor3f(0.3, 0.3, 0.3)  # Darker color for contrast
        
        # Draw grid outline
        glVertex3f(0, 0, 0)
        glVertex3f(COLS, 0, 0)
        
        glVertex3f(0, 0, 0)
        glVertex3f(0, ROWS, 0)
        
        glVertex3f(0, 0, 0)
        glVertex3f(0, 0, LAYERS)
        
        glVertex3f(COLS, 0, 0)
        glVertex3f(COLS, ROWS, 0)
        
        glVertex3f(COLS, 0, 0)
        glVertex3f(COLS, 0, LAYERS)
        
        glVertex3f(0, ROWS, 0)
        glVertex3f(COLS, ROWS, 0)
        
        glVertex3f(0, ROWS, 0)
        glVertex3f(0, ROWS, LAYERS)
        
        glVertex3f(0, 0, LAYERS)
        glVertex3f(COLS, 0, LAYERS)
        
        glVertex3f(0, 0, LAYERS)
        glVertex3f(0, ROWS, LAYERS)
        
        glVertex3f(COLS, ROWS, 0)
        glVertex3f(COLS, ROWS, LAYERS)
        
        glVertex3f(COLS, 0, LAYERS)
        glVertex3f(COLS, ROWS, LAYERS)
        
        glVertex3f(0, ROWS, LAYERS)
        glVertex3f(COLS, ROWS, LAYERS)
        
        glEnd()
        glLineWidth(1.0)  # Reset line width
        
        # Re-enable lighting
        glEnable(GL_LIGHTING)
    
    
    
    def draw_target(self):
        # Temporarily disable lighting for rotating indicators
        glDisable(GL_LIGHTING)
        
        # Draw target position indicator before the target itself
        glPushMatrix()
        glTranslatef(self.target_x, self.target_y, self.target_z)
        
        # Draw a vertical line to make target more visible
        glLineWidth(3.0)
        glBegin(GL_LINES)
        glColor3f(*GL_YELLOW)
        glVertex3f(0, -5, 0)
        glVertex3f(0, self.target_y * 2, 0)
        glEnd()
        glLineWidth(1.0)
        
        # Add a rotating indicator for visibility
        glRotatef(self.simulation_time * 100, 0, 1, 0)
        
        glLineWidth(3.0)
        glBegin(GL_LINES)
        glColor3f(*GL_YELLOW)
        line_length = 1.5  # Longer lines
        glVertex3f(0, 0, 0)
        glVertex3f(line_length, 0, 0)
        glVertex3f(0, 0, 0)
        glVertex3f(0, line_length, 0)
        glVertex3f(0, 0, 0)
        glVertex3f(0, 0, line_length)
        glEnd()
        glLineWidth(1.0)
        
        glPopMatrix()
        
        # Re-enable lighting for the target sphere
        glEnable(GL_LIGHTING)
        
        # Draw target as a sphere
        glPushMatrix()
        glTranslatef(self.target_x, self.target_y, self.target_z)
        
        # Draw target as a sphere
        glColor3f(*GL_RED)
        quadric = gluNewQuadric()
        gluSphere(quadric, 0.8, 16, 16)  # Larger target sphere
        gluDeleteQuadric(quadric)
        
        glPopMatrix()
    
    def draw_grid(self):
        # Draw grid structure - only draw specific cells based on their state
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                for z in range(self.grid.depth):
                    node = self.grid.get_node(x, y, z)
                    
                    # Only draw cells that are:
                    # 1. Obstacles
                    # 2. Visited by drones
                    # 3. Target position
                    # 4. Cells with high exploration value
                    if not node.walkable:
                        # For obstacles, only draw those that are near explored areas or on the boundary
                        has_explored_neighbor = False
                        for dx, dy, dz in [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]:
                            nx, ny, nz = x + dx, y + dy, z + dz
                            if (0 <= nx < self.grid.width and 
                                0 <= ny < self.grid.height and 
                                0 <= nz < self.grid.depth):
                                neighbor = self.grid.get_node(nx, ny, nz)
                                if neighbor.scanned:
                                    has_explored_neighbor = True
                                    break
                        
                        # Only draw obstacle if it's near explored area
                        if has_explored_neighbor:
                            self.draw_cell(x, y, z, node)
                    elif (x, y, z) == (self.target_x, self.target_y, self.target_z):
                        # Always draw target
                        self.draw_cell(x, y, z, node)
                    elif node.visited_by:
                        # Always draw visited cells
                        self.draw_cell(x, y, z, node)
                    elif node.scanned:
                        # For scanned cells, only draw a fraction to reduce clutter
                        # The closer to a drone path, the more likely to draw
                        draw_probability = 0.3  # Base probability
                        
                        # Increase probability if near a drone
                        for drone in self.drones:
                            dist = math.sqrt((x - drone.x)**2 + (y - drone.y)**2 + (z - drone.z)**2)
                            if dist < EXPLORATION_RADIUS:
                                draw_probability += 0.4 * (1 - dist/EXPLORATION_RADIUS)
                        
                        if random.random() < draw_probability:
                            self.draw_cell(x, y, z, node)

def draw_drone(self):
        # Calculate interpolated position based on move_fraction
        display_x = self.x
        display_y = self.y
        display_z = self.z
        
        if self.move_fraction > 0 and self.path:
            next_x, next_y, next_z = self.path[0]
            display_x = self.x + (next_x - self.x) * self.move_fraction
            display_y = self.y + (next_y - self.y) * self.move_fraction
            display_z = self.z + (next_z - self.z) * self.move_fraction
        
        # Temporarily disable lighting for paths
        glDisable(GL_LIGHTING)
        
        # Draw path
        if self.path and len(self.path) > 0:
            glLineWidth(2.0)  # Thicker path lines
            glBegin(GL_LINE_STRIP)
            glColor3f(*self.color)
            
            # Start from current position
            glVertex3f(display_x, display_y, display_z)
            
            # Add all path points
            for x, y, z in self.path:
                glVertex3f(x, y, z)
            
            glEnd()
            glLineWidth(1.0)  # Reset line width
        
        # Re-enable lighting for drone body
        glEnable(GL_LIGHTING)
        
        # Draw the drone as a sphere
        glPushMatrix()
        glTranslatef(display_x, display_y, display_z)
        
        # Set drone color (more vibrant)
        glColor3f(*self.color)
        
        # Draw drone body (sphere)
        quadric = gluNewQuadric()
        gluSphere(quadric, DRONE_RADIUS * 1.5, 16, 16)  # Increased radius and segments
        gluDeleteQuadric(quadric)
        
        # Disable lighting for exploration radius
        glDisable(GL_LIGHTING)
        
        # Draw exploration radius (wireframe sphere)
        if self.id == 0:  # Only draw for first drone to reduce visual clutter
            glColor4f(*self.color, 0.3)  # Transparent
            glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
            
            quadric = gluNewQuadric()
            gluSphere(quadric, EXPLORATION_RADIUS, 12, 12)
            gluDeleteQuadric(quadric)
            
            glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)
        
        glPopMatrix()
        
        # Re-enable lighting
        glEnable(GL_LIGHTING)
    
        
# Add a function for calculating search efficiency metrics
def calculate_efficiency(drones, simulation_time, total_cells, scanned_cells):
    """Calculate efficiency metrics for the search operation."""
    coverage_percent = (scanned_cells / total_cells) * 100
    cells_per_second = scanned_cells / max(simulation_time, 0.001)  # Avoid division by zero
    cells_per_drone = scanned_cells / len(drones)
    
    return {
        "coverage_percent": coverage_percent,
        "cells_per_second": cells_per_second,
        "cells_per_drone": cells_per_drone,
        "search_time": simulation_time
    }

# Add a results display function
def display_results(game):
    """Display results after simulation is complete."""
    # Calculate final metrics
    total_cells = game.grid.width * game.grid.height * game.grid.depth
    scanned_cells = sum(1 for x in range(game.grid.width) 
                        for y in range(game.grid.height) 
                        for z in range(game.grid.depth) 
                        if game.grid.get_node(x, y, z).scanned)
    
    metrics = calculate_efficiency(game.drones, game.simulation_time, total_cells, scanned_cells)
    
    # Create a results window
    results_surface = pygame.Surface((400, 300))
    results_surface.fill(DARK_GRAY)
    font = pygame.font.SysFont(None, 28)
    
    # Display results
    lines = [
        "Search Results",
        f"Target found: {game.target_found}",
        f"Time taken: {game.simulation_time:.2f} seconds",
        f"Coverage: {metrics['coverage_percent']:.1f}%",
        f"Cells scanned per second: {metrics['cells_per_second']:.1f}",
        f"Cells per drone: {metrics['cells_per_drone']:.1f}",
        "",
        "Press R to restart or ESC to quit"
    ]
    
    for i, line in enumerate(lines):
        text = font.render(line, True, WHITE)
        results_surface.blit(text, (20, 20 + i * 30))
    
    # Display on screen
    game.screen.blit(results_surface, (WIDTH//2 - 200, HEIGHT//2 - 150))
    pygame.display.flip()
    
    # Wait for restart or quit
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    return False
                elif event.key == pygame.K_r:
                    return True
    
    return False

# Main function to run the game
def main():
    running = True
    while running:
        game = Game()
        game.run()
        
        # Show results and check if we should restart
        running = display_results(game)

if __name__ == "__main__":
    main()

In [ ]:
# FLocking herd
import pygame
import numpy as np
import math
import random
from pygame import Vector2

# Ekran ayarları
WIDTH, HEIGHT = 1000, 700
BACKGROUND_COLOR = (10, 10, 40)

# Sürü parametreleri
NUM_BOIDS = 50
BOID_SIZE = 6
BOID_COLOR = (220, 220, 255)
LEADER_COLOR = (255, 100, 100)
PERCEPTION_RADIUS = 100
MAX_SPEED = 4.0
MIN_SPEED = 2.0

# Herd prensibi ağırlıkları
COHESION_WEIGHT = 0.005  # Merkeze doğru hareket
ALIGNMENT_WEIGHT = 0.1   # Hizalanma
SEPARATION_WEIGHT = 0.2  # Ayrılma
BOUNDARY_WEIGHT = 0.08   # Sınır kontrolü
OBSTACLE_WEIGHT = 0.3    # Engel kaçınması

# Engel ayarları
NUM_OBSTACLES = 3
OBSTACLE_COLOR = (200, 50, 50)
OBSTACLE_SIZES = [(70, 70), (90, 90), (50, 50)]
MIN_OBSTACLE_DISTANCE = 150  # Engeller arası minimum mesafe
OBSTACLE_AVOIDANCE_RADIUS = 100  # Engelden kaçınma yarıçapı

class Boid:
    def __init__(self, x, y, is_leader=False):
        self.position = Vector2(x, y)
        self.velocity = Vector2(random.uniform(-1, 1), random.uniform(-1, 1)).normalize() * random.uniform(MIN_SPEED, MAX_SPEED)
        self.acceleration = Vector2(0, 0)
        self.max_force = 0.3
        self.max_speed = MAX_SPEED
        self.min_speed = MIN_SPEED
        self.is_leader = is_leader
        # Liderler biraz daha hızlı olabilir
        if is_leader:
            self.max_speed = MAX_SPEED * 1.2
    
    def update(self):
        # Hızı güncelle
        self.velocity += self.acceleration
        # Hız sınırlaması
        speed = self.velocity.length()
        if speed > self.max_speed:
            self.velocity = self.velocity.normalize() * self.max_speed
        elif speed < self.min_speed:
            self.velocity = self.velocity.normalize() * self.min_speed
        
        # Pozisyonu güncelle
        self.position += self.velocity
        # İvmeyi sıfırla
        self.acceleration = Vector2(0, 0)
        
        # Ekranın diğer tarafına geçme (toroidal)
        if self.position.x > WIDTH:
            self.position.x = 0
        elif self.position.x < 0:
            self.position.x = WIDTH
        if self.position.y > HEIGHT:
            self.position.y = 0
        elif self.position.y < 0:
            self.position.y = HEIGHT
    
    def apply_force(self, force):
        # F = ma, m=1 olduğu için F = a
        self.acceleration += force
    
    def seek(self, target):
        # Hedefe doğru yönlendirme vektörü
        desired = target - self.position
        
        # Mesafe hesapla
        distance = desired.length()
        
        if distance > 0:
            # Maksimum hızda hareket etmek istiyoruz
            desired = desired.normalize() * self.max_speed
            
            # Reynolds'un steering formülü: istenen - mevcut
            steering = desired - self.velocity
            
            # Force'u sınırla
            if steering.length() > self.max_force:
                steering = steering.normalize() * self.max_force
                
            return steering
        else:
            return Vector2(0, 0)
    
    def cohesion(self, boids):
        # Gruba doğru yönlendirme (boids'in ortalamasına doğru)
        steering = Vector2(0, 0)
        total = 0
        avg_position = Vector2(0, 0)
        
        # Algılama yarıçapı içindeki diğer boid'leri bul
        for other in boids:
            distance = self.position.distance_to(other.position)
            if other != self and distance < PERCEPTION_RADIUS:
                avg_position += other.position
                total += 1
        
        if total > 0:
            avg_position /= total
            # Ortalama pozisyona doğru yönlendirme
            return self.seek(avg_position) * COHESION_WEIGHT
        else:
            return Vector2(0, 0)
    
    def alignment(self, boids):
        # Grubun yönü ile hizalanma
        steering = Vector2(0, 0)
        total = 0
        avg_velocity = Vector2(0, 0)
        
        # Algılama yarıçapı içindeki diğer boid'leri bul
        for other in boids:
            distance = self.position.distance_to(other.position)
            if other != self and distance < PERCEPTION_RADIUS:
                avg_velocity += other.velocity
                total += 1
        
        if total > 0:
            avg_velocity /= total
            # Ortalama hıza doğru yönlendirme
            steering = avg_velocity - self.velocity
            if steering.length() > self.max_force:
                steering = steering.normalize() * self.max_force
            
            return steering * ALIGNMENT_WEIGHT
        else:
            return Vector2(0, 0)
    
    def separation(self, boids):
        # Diğer boid'lerden uzaklaşma
        steering = Vector2(0, 0)
        total = 0
        
        for other in boids:
            distance = self.position.distance_to(other.position)
            if other != self and distance < PERCEPTION_RADIUS / 2:  # Daha yakın mesafede
                # Diğer boid'den uzaklaşma vektörü
                diff = self.position - other.position
                if diff.length() > 0:
                    # Mesafe ile ters orantılı etki (yakın olanlardan daha hızlı kaç)
                    diff = diff.normalize() / max(distance, 0.1)
                    steering += diff
                    total += 1
        
        if total > 0:
            steering /= total
            if steering.length() > 0:
                steering = steering.normalize() * self.max_speed
                steering -= self.velocity
                if steering.length() > self.max_force:
                    steering = steering.normalize() * self.max_force
            
            return steering * SEPARATION_WEIGHT
        else:
            return Vector2(0, 0)
    
    def avoid_obstacles(self, obstacles):
        # Engellerden kaçınma
        steering = Vector2(0, 0)
        
        for obstacle in obstacles:
            # Engel ve boid arasındaki mesafe
            obstacle_center = Vector2(obstacle[0] + obstacle[2]/2, obstacle[1] + obstacle[3]/2)
            distance = self.position.distance_to(obstacle_center) - (obstacle[2] + obstacle[3])/4  # Ortalama yarıçap
            
            if distance < OBSTACLE_AVOIDANCE_RADIUS:
                # Engelden uzaklaşma vektörü
                diff = self.position - obstacle_center
                if diff.length() > 0:
                    # Mesafe ile ters orantılı etki
                    repulsion = diff.normalize() * (OBSTACLE_AVOIDANCE_RADIUS / max(distance, 1))
                    steering += repulsion
        
        if steering.length() > 0:
            if steering.length() > self.max_force:
                steering = steering.normalize() * self.max_force
            
            return steering * OBSTACLE_WEIGHT
        else:
            return Vector2(0, 0)
    
    def keep_within_bounds(self):
        # Ekranın kenarlarından kaçınma
        steering = Vector2(0, 0)
        border = 50
        
        if self.position.x < border:
            steering.x = border - self.position.x
        elif self.position.x > WIDTH - border:
            steering.x = (WIDTH - border) - self.position.x
            
        if self.position.y < border:
            steering.y = border - self.position.y
        elif self.position.y > HEIGHT - border:
            steering.y = (HEIGHT - border) - self.position.y
        
        if steering.length() > 0:
            steering = steering.normalize() * self.max_speed
            steering -= self.velocity
            if steering.length() > self.max_force:
                steering = steering.normalize() * self.max_force
            
            return steering * BOUNDARY_WEIGHT
        else:
            return Vector2(0, 0)
    
    def follow_leader(self, leader, boids):
        # Lideri takip etme davranışı
        if self.is_leader or leader is None:
            return Vector2(0, 0)
        
        distance = self.position.distance_to(leader.position)
        
        if distance < PERCEPTION_RADIUS * 1.5:
            # Liderin yakınındayız, normal sürü davranışlarını uygula
            return Vector2(0, 0)
        else:
            # Lidere doğru yönlenme
            return self.seek(leader.position) * 0.1
    
    def flock(self, boids, obstacles, leader=None):
        # Tüm sürü davranışlarını uygula
        cohesion_force = self.cohesion(boids)
        alignment_force = self.alignment(boids)
        separation_force = self.separation(boids)
        boundary_force = self.keep_within_bounds()
        obstacle_force = self.avoid_obstacles(obstacles)
        
        # Lider ise, kendi yolunu belirler (daha az etkilenir)
        if self.is_leader:
            # Liderler daha bağımsız hareket eder
            self.apply_force(cohesion_force * 0.3)
            self.apply_force(alignment_force * 0.3)
            self.apply_force(separation_force)
            self.apply_force(boundary_force)
            self.apply_force(obstacle_force)
        else:
            # Normal boid davranışları
            leader_force = self.follow_leader(leader, boids)
            self.apply_force(cohesion_force)
            self.apply_force(alignment_force)
            self.apply_force(separation_force)
            self.apply_force(boundary_force)
            self.apply_force(obstacle_force)
            self.apply_force(leader_force)
    
    def draw(self, screen):
        # Boid'i çiz (üçgen şeklinde, hareket yönünde)
        color = LEADER_COLOR if self.is_leader else BOID_COLOR
        
        # Dönüş açısını hesapla
        angle = math.atan2(self.velocity.y, self.velocity.x)
        
        # Üçgen köşeleri (burun, sağ kanat, sol kanat)
        size = BOID_SIZE * 1.5 if self.is_leader else BOID_SIZE
        tip = (self.position.x + math.cos(angle) * size * 2, 
               self.position.y + math.sin(angle) * size * 2)
        right = (self.position.x + math.cos(angle + 2.5) * size, 
                 self.position.y + math.sin(angle + 2.5) * size)
        left = (self.position.x + math.cos(angle - 2.5) * size, 
                self.position.y + math.sin(angle - 2.5) * size)
        
        # Üçgeni çiz
        pygame.draw.polygon(screen, color, [tip, right, left])
        
        # Algılama yarıçapını göster (DEBUG)
        # if self.is_leader:
        #     pygame.draw.circle(screen, (100, 100, 100, 50), 
        #                       (int(self.position.x), int(self.position.y)), 
        #                       PERCEPTION_RADIUS, 1)


def generate_obstacles(num_obstacles):
    obstacles = []
    for i in range(num_obstacles):
        while True:
            # Rastgele pozisyon oluştur
            size = OBSTACLE_SIZES[i % len(OBSTACLE_SIZES)]
            x = random.randint(size[0], WIDTH - size[0])
            y = random.randint(size[1], HEIGHT - size[1])
            
            # Diğer engellerle çakışma kontrolü
            valid_position = True
            for other in obstacles:
                other_center = Vector2(other[0] + other[2]/2, other[1] + other[3]/2)
                this_center = Vector2(x + size[0]/2, y + size[1]/2)
                distance = this_center.distance_to(other_center)
                
                if distance < MIN_OBSTACLE_DISTANCE:
                    valid_position = False
                    break
            
            if valid_position:
                obstacles.append((x, y, size[0], size[1]))
                break
    
    return obstacles


def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Herd Prensibi Simülasyonu")
    clock = pygame.time.Clock()
    
    # Font ayarları
    font = pygame.font.SysFont('Arial', 18)
    
    # Engelleri oluştur
    obstacles = generate_obstacles(NUM_OBSTACLES)
    
    # Boid'leri oluştur
    boids = []
    # Bir lider oluştur
    leader = Boid(WIDTH/2, HEIGHT/2, is_leader=True)
    boids.append(leader)
    
    # Diğer boid'leri oluştur
    for i in range(NUM_BOIDS - 1):
        x = random.randint(0, WIDTH)
        y = random.randint(0, HEIGHT)
        boids.append(Boid(x, y))
    
    running = True
    paused = False
    show_debug = False
    show_help = True
    
    # Yardım mesajı
    help_text = [
        "KONTROLLER:",
        "SPACE: Duraklat/Devam Et",
        "D: Debug bilgileri",
        "H: Yardımı göster/gizle",
        "ESC: Çıkış",
        "",
        "SÜRİ PARAMETRELERİ:",
        f"Boid Sayısı: {NUM_BOIDS}",
        f"Algılama Yarıçapı: {PERCEPTION_RADIUS}",
        f"Kohezyon: {COHESION_WEIGHT}",
        f"Hizalama: {ALIGNMENT_WEIGHT}",
        f"Ayrılma: {SEPARATION_WEIGHT}"
    ]
    
    while running:
        # Olayları işle
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_SPACE:
                    paused = not paused
                elif event.key == pygame.K_d:
                    show_debug = not show_debug
                elif event.key == pygame.K_h:
                    show_help = not show_help
        
        if not paused:
            # Boid'leri güncelle
            for boid in boids:
                boid.flock(boids, obstacles, leader)
            
            for boid in boids:
                boid.update()
        
        # Ekranı temizle
        screen.fill(BACKGROUND_COLOR)
        
        # Engelleri çiz
        for obstacle in obstacles:
            pygame.draw.rect(screen, OBSTACLE_COLOR, obstacle)
        
        # Boid'leri çiz
        for boid in boids:
            boid.draw(screen)
        
        # Debug bilgilerini göster
        if show_debug:
            fps_text = font.render(f"FPS: {int(clock.get_fps())}", True, (200, 200, 200))
            screen.blit(fps_text, (10, 10))
            
            # Boid sayısını göster
            boid_count_text = font.render(f"Boid Sayısı: {len(boids)}", True, (200, 200, 200))
            screen.blit(boid_count_text, (10, 30))
        
        # Yardım metnini göster
        if show_help:
            for i, line in enumerate(help_text):
                help_line = font.render(line, True, (180, 180, 180))
                screen.blit(help_line, (WIDTH - 250, 20 + i * 22))
        
        # Ekranı güncelle
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()


if __name__ == "__main__":
    main()